In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from catboost import CatBoostClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

C:\Users\royal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
##for printing multiple output..
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse.linalg import svds
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

from surprise import Reader, Dataset, SVD, evaluate

C:\Users\royal\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


### Read Csv

In [227]:
os.chdir(r'C:\Users\royal\Downloads\bb020fc4-9-Dataset\Dataset\training dataset')

In [228]:
##main user or account
account = pd.read_csv('Accounts.csv')
##interaction files
acc_prop = pd.read_csv('Accounts_properties.csv')
deal_prop = pd.read_csv('Deals_to_Properties.csv')
oppor = pd.read_csv('Opportunities.csv')

In [6]:
##test dataset have detail records in account table.
os.chdir(r'C:\Users\royal\Downloads\bb020fc4-9-Dataset\Dataset\Test dataset')
test  = pd.read_csv('Test_Data.csv')

In [208]:
##main item or movie file
os.chdir(r'C:\Users\royal\Downloads\bb020fc4-9-Dataset\Dataset\Property dataset')
propert = pd.read_csv('Properties.csv')

In [214]:
propert = pd.read_csv('property_rem.csv')

In [346]:
##property removed after findings from submission

In [14]:
#415 properties removed
propert = propert[propert.id_props.isin(list(set(propert.id_props.unique()).intersection(deal_prop.id_props.unique())))]

In [180]:
propert.shape

(9386, 24)

In [18]:
##1033 properties removed who's built year is before 1985
propert[(propert.year_built <= 1985)].shape
propert = propert[~(propert.year_built <= 1985)]
# propert[propert.year_built<=1950].shape

(6500, 24)

In [26]:
##195 removed failed submission
propert[propert.id_props.isin(list(set(subm.id_prop.unique())))].shape
propert = propert[~propert.id_props.isin(list(set(subm.id_prop.unique())))]

(214, 24)

In [20]:
##50 demolished property removed
## proposed,under construction and existing status building can reccomended
propert = propert[~(propert.building_status == 'Demolished')]

In [22]:
##167 removed who's sale year is before 2003
propert.shape
propert = propert[~(propert.sale_year < 2003)]

(11339, 24)

In [44]:
propert[(propert.sale_status == 'Y')].shape
propert = propert[~(propert.sale_status == 'Y')]

(1000, 24)

In [177]:
propert[(propert.id_props.isin(list(set(ll1))))]
propert = propert[~(propert.id_props.isin(list(set(ll1))))]

id_props     building_status  \
206   a0I2A00000XQGSXUA5            Existing   
888   a0I2A00000XQGvOUAX            Existing   
930   a0I2A00000XQGx3UAH            Existing   
958   a0I2A00000XQGyPUAX                 NaN   
971   a0I2A00000XQGzBUAX                 NaN   
1033  a0I2A00000XQH1qUAH            Existing   
1034  a0I2A00000XQH1sUAH            Existing   
1089  a0I2A00000XQH4eUAH            Existing   
1291  a0I2A00000XQHDaUAP            Existing   
1298  a0I2A00000XQHDrUAP            Existing   
1671  a0I2A00000XQHUIUA5                 NaN   
1697  a0I2A00000XQHVeUAP                 NaN   
1783  a0I2A00000XQHZlUAP            Existing   
1893  a0I2A00000XQHfAUAX            Existing   
2069  a0I2A00000XQHmzUAH                 NaN   
2102  a0I2A00000XQHoJUAX                 NaN   
2144  a0I2A00000XQHq6UAH                 NaN   
2283  a0I2A00000XQHwaUAH                 NaN   
2284  a0I2A00000XQHwbUAH                 NaN   
2376  a0I2A00000XQI0HUAX            Existing   
2388  a0I2A00000XQI0sUAH            Existing   
2469  a0I2A00000XQI4pUAH            Existing   
2614  a0I2A00000XQIAAUA5            Existing   
2619  a0I2A00000XQIAIUA5                 NaN   
2783  a0I2A00000XQIHgUAP            Existing   
2982  a0I2A00000XQIQbUAP                 NaN   
3217  a0I2A00000XQIbMUAX                 NaN   
3248  a0I2A00000XQIcjUAH            Existing   
3356  a0I2A00000XQIhxUAH            Existing   
3366  a0I2A00000XQIiMUAX            Existing   
...                  ...                 ...   
5375  a0I2A00000XQKBPUA5            Existing   
5520  a0I2A00000XQKHlUAP                 NaN   
5944  a0I2A00000XQKYxUAP            Existing   
5985  a0I2A00000XQKaJUAX            Existing   
6276  a0I2A00000XQKlfUAH            Existing   
6308  a0I2A00000XQKmtUAH            Existing   
6417  a0I2A00000XQKrUUAX            Existing   
6751  a0I2A00000XQL5CUAX            Existing   
6761  a0I2A00000XQL5dUAH            Existing   
6818  a0I2A00000XQL8TUAX  Under Construction   
7095  a0I2A00000XQLKeUAP                 NaN   
7248  a0I2A00000XQLRmUAP                 NaN   
7251  a0I2A00000XQLRwUAP                 NaN   
7328  a0I2A00000XQLVMUA5                 NaN   
7366  a0I2A00000XQLX3UAP            Existing   
7448  a0I2A00000XQLaqUAH            Existing   
7618  a0I2A00000XQLhuUAH                 NaN   
7693  a0I2A00000XQLl6UAH            Existing   
7743  a0I2A00000XQLncUAH                 NaN   
7745  a0I2A00000XQLniUAH            Existing   
7845  a0I2A00000XQLsJUAX                 NaN   
7846  a0I2A00000XQLsKUAX                 NaN   
7982  a0I2A00000XQLyBUAX            Existing   
8084  a0I2A00000XQM2UUAX            Existing   
8085  a0I2A00000XQM2VUAX            Existing   
8127  a0I2A00000XQM4OUAX            Existing   
8131  a0I2A00000XQM4UUAX            Existing   
8236  a0I2A00000XQM9UUAX                 NaN   
8250  a0I2A00000XQMAJUA5                 NaN   
8897  a0I2A00000Y68eLUAR                 NaN   

                                  building_tax_expenses       city    class  \
206   2016 Tax @ $14.43/sf, 2015 Est Tax @ $13.67/sf...  City-0376  Class A   
888                                2015 Tax @ $40.25/sf  City-0070      NaN   
930   2015 Tax @ $3026.53/Unit; 2013 Ops @ $3263.36/...  City-0725  Class A   
958                                                 NaN  City-0323      NaN   
971                                                 NaN  City-0019  Class A   
1033                  2005 Combined Tax/Ops @ $21.38/sf  City-0090  Class A   
1034                               2009 Tax @ $95.44/sf  City-1673  Class B   
1089          2006 Tax @ $4.77/sf; 2007 Ops @ $12.72/sf  City-1606  Class A   
1291                                2015 Tax @ $1.93/sf  City-0960  Class A   
1298                                2016 Tax @ $3.75/sf  City-0970  Class A   
1671                                                NaN  City-1212  Class A   
1697                                                NaN  City-1415  Class A   
1783  2015 T

In [178]:
propert.to_csv('property_rem.csv',index=False)

## cleanup

In [10]:
##same person involved in all deals.
deal_prop.drop('createdbyid', axis=1, inplace=True)

In [57]:
##id_props & id_deals are same.
propert.drop('id_deals', axis=1, inplace=True)

In [13]:
##id & id_deals were same
oppor.drop(oppor.columns[0], axis=1, inplace=True)

In [14]:
deal_prop.to_csv('Deals_to_Properties.csv',index=False)
acc_prop.to_csv('Accounts_properties.csv',index=False)
oppor.to_csv('Opportunities.csv',index=False)

In [58]:
os.chdir(r'C:\Users\royal\Downloads\bb020fc4-9-Dataset\Dataset\Property dataset')
propert.to_csv('Properties.csv',index=False)

In [12]:
oppor.columns[0]

'id_deals'

## Exploration

#### Accounts: This table has information on customers/accounts. These are the accounts of whom we are marketing the properties for sale.

In [8]:
account.head()

id_accs  active_deals  activity_count  buyer_book  \
0  0012A000023XlCCQA0             1            2367        True   
1  0012A000023XmOxQAK             9            7974        True   
2  0012A000023YLqwQAG           891           50161        True   
3  0012A000023XlNjQAK            34           14631        True   
4  0012A000023YBFRQA4           283          194195        True   

   servicing_contract               investor_type   cmbs  consultant  \
0               False  Private Investor/Developer  False       False   
1               False                        REIT  False       False   
2                True               Bank / Thrift  False       False   
3               False                Conduit/CMBS   True       False   
4               False      Life Insurance Company  False       False   

   correspondent  foreign  master_servicer  lender_book  loan_sales_book  \
0          False    False            False        False            False   
1          False    False            False         True             True   
2          False    False            False         True             True   
3          False     True            False         True             True   
4          False    False            False         True             True   

   loan_servicing  num_deals_as_client  num_deals_as_investor  \
0           False                    0                      1   
1           False                    7                      2   
2           False                   40                    851   
3           False                    0                     34   
4           False                   63                    220   

   number_of_properties  number_of_related_deals  \
0                     1                        2   
1                    77                       18   
2                   693                      996   
3                     0                       45   
4                   393                      383   

   number_of_related_properties  number_of_won_deals_as_client  
0                             1                              0  
1                            85                              7  
2                          1167                             37  
3                            14                              0  
4                           591                             46

In [9]:
account.shape    ##total account id's including present in test.
account['active_deals'].sum()  ##can represent number of record present in some of available file.

(2756, 20)

45368

#### Accounts to Properties: This table comprises information on properties that have been already bought by the accounts.Account information and the property details of the lead

In [10]:
acc_prop.head()

id             id_accs            id_props
0  a062A00001Whq5tQAB  0012A000023XlCCQA0  a0I2A00000XQKwXUAX
1  a062A00001VmP7EQAV  0012A000023XmOxQAK  a0I2A00000XvUEwUAN
2  a062A00001VmQByQAN  0012A000023XmOxQAK  a0I2A00000XwQ13UAF
3  a062A00001VmQC2QAN  0012A000023XmOxQAK  a0I2A00000XwZyBUAV
4  a062A00001VmW8wQAF  0012A000023XmOxQAK  a0I2A00000ZGZClUAP

In [11]:
acc_prop.shape
acc_prop['id'].nunique()  ##unique id
acc_prop['id_accs'].nunique()   ##train id for which props are given
acc_prop['id_props'].nunique()  ##need to find out how it has more property id's then property file??

(70350, 3)

70350

2727

63001

In [48]:
##acc id's which are in both account and account_to property table.
len(set(acc_prop.id_accs.unique()).intersection(account.id_accs.unique()))
##acc id's which are in both test and account_to property table. found zero so no record for account to property for test id's
len(set(test.id_accs.unique()).intersection(acc_prop.id_accs.unique()))
##property id's which are in both property and account_to property table.
len(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))
propert[propert.sale_status == 'Y'].shape

2727

0

8121

(0, 24)

In [49]:
##for property to account toproperty mapping
propert[propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))))].shape
##forproperty to deal toproperty mapping
propert[propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(deal_prop.id_props.unique()))))].shape

qa = propert[~propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(deal_prop.id_props.unique()))))]
qa[qa.sale_status == 'N'].shape
len(set(qa[qa.sale_status == 'N'].id_props.unique()).intersection(acc_prop.id_props.unique()))

qa = propert[~propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))))]
qa.shape
qa[qa.sale_status == 'N'].shape
len(set(qa[qa.sale_status == 'N'].id_props.unique()).intersection(deal_prop.id_props.unique()))

(8121, 24)

(9974, 24)

(0, 24)

0

(1853, 24)

(1853, 24)

1853

In [54]:
aa = set(deal_prop.id_props.unique()).intersection(acc_prop.id_props.unique())
propert[~propert.id_props.isin(aa.intersection(set(propert.id_props.unique())))].shape

(1853, 24)

In [53]:
###these 21 accounts are not reccomended when status='Y'
qa = propert[propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))))]
qa[qa.sale_status == 'N'].shape
len(set(qa[qa.sale_status == 'N'].id_props.unique()).intersection(deal_prop.id_props.unique()))

l1 = qa[qa.sale_status == 'Y'].id_props.values
##these 415 properties not reccomended so removed
#propert[~propert.id_props.isin(list(set(propert.id_props.unique()).intersection(deal_prop.id_props.unique())))]
qa = propert[~propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))))]
qa.shape
# l1 = propert[propert.sale_status == 'Y'].id_props.values
# l1 = propert[propert.year_built < 1985].id_props.values
# l1 = propert[(propert.building_status == 'Under Construction')].id_props.values
l1 = propert[~propert.id_props.isin(list(set(propert[propert.sale_status == 'N'].id_props.unique()).intersection(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))))].id_props.values

# aa = set(deal_prop.id_props.unique()).intersection(acc_prop.id_props.unique())
# l1 = propert[~propert.id_props.isin(aa.intersection(set(propert.id_props.unique())))].id_props.values
len(l1)
sub_file_all_same(l1)

(8121, 24)

8121

(1853, 24)

1853

53737

In [235]:
propert[propert.id_props.isin(acc_prop[acc_prop.id_accs =='0012A000023XnJ0QAK'].id_props)].head()
propert[propert.id_props.isin(deal_prop[deal_prop.id_deals.isin(oppor[oppor.id_accs =='0012A000023XnJ0QAK'].id_deals)].id_props)].head(10)

# propert[propert.sale_date__c]
# propert['sale_year'] = pd.DatetimeIndex(propert['sale_date__c']).year

id_props building_status building_tax_expenses       city  \
3812  a0I2A00000XQJ40UAH             NaN                   NaN  City-1510   
6591  a0I2A00000XQL0lUAH             NaN                   NaN  City-2059   
6844  a0I2A00000XQLBvUAP             NaN                   NaN  City-1779   
8488  a0I2A00000XucBlUAJ             NaN                   NaN  City-1852   

     class       county sale_status  portfolio       market  num_buildings  \
3812   NaN  County-0464           N      False  Market-0169           15.0   
6591   NaN  County-0630           N      False  Market-0002            1.0   
6844   NaN  County-0503           N      False  Market-0138            NaN   
8488   NaN  County-0124           N      False  Market-0212            1.0   

        ...      price_per_unit  property_type_1  region__c  sale_amount__c  \
3812    ...                 NaN           Retail  Southwest      29050000.0   
6591    ...                 NaN           Retail  Southeast       7043000.0   
6844    ...                 NaN           Retail  Southeast      22450000.0   
8488    ...                 NaN           Retail  Southeast      12150000.0   

      sale_date__c size_acres   size_sf  size_units year_built  sale_year  
3812     1/31/2017     22.210  241831.0         NaN     2006.0     2017.0  
6591    11/14/2013      5.730   52600.0         NaN     2012.0     2013.0  
6844      2/3/2017     31.205  298755.0         NaN     1992.0     2017.0  
8488     7/12/2017      9.000   87349.0         NaN     1996.0     2017.0  

[4 rows x 24 columns]

id_props building_status building_tax_expenses       city  \
805   a0I2A00000XQGsEUAX             NaN                   NaN  City-0396   
3230  a0I2A00000XQIdNUAX             NaN                   NaN  City-1595   
6275  a0I2A00000XQKnzUAH             NaN                   NaN  City-0413   
6591  a0I2A00000XQL0lUAH             NaN                   NaN  City-2059   
8043  a0I2A00000XQM3wUAH             NaN                   NaN  City-2096   
8488  a0I2A00000XucBlUAJ             NaN                   NaN  City-1852   

     class       county sale_status  portfolio       market  num_buildings  \
805    NaN  County-0431           N      False  Market-0050            NaN   
3230   NaN  County-0217           N      False  Market-0175            NaN   
6275   NaN  County-0291           N      False  Market-0011            NaN   
6591   NaN  County-0630           N      False  Market-0002            1.0   
8043   NaN  County-0370           N      False  Market-0082            NaN   
8488   NaN  County-0124           N      False  Market-0212            1.0   

        ...      price_per_unit  property_type_1     region__c  \
805     ...                 NaN           Retail     Southeast   
3230    ...                 NaN           Retail     Southeast   
6275    ...                 NaN           Retail  Mid-Atlantic   
6591    ...                 NaN           Retail     Southeast   
8043    ...                 NaN           Retail     Southeast   
8488    ...                 NaN           Retail     Southeast   

      sale_amount__c  sale_date__c size_acres   size_sf  size_units  \
805        6154908.0      7/9/2013      23.24  124260.0         NaN   
3230       7100000.0    11/30/2006      14.70  165851.0         NaN   
6275             NaN           NaN        NaN   66700.0         NaN   
6591       7043000.0    11/14/2013       5.73   52600.0         NaN   
8043             NaN           NaN       8.18   74947.0         NaN   
8488      12150000.0     7/12/2017       9.00   87349.0         NaN   

     year_built  sale_year  
805      1986.0     2013.0  
3230     1988.0     2006.0  
6275     1990.0        NaN  
6591     2012.0     2013.0  
8043     2004.0        NaN  
8488     1996.0     2017.0  

[6 rows x 24 columns]

In [412]:
propert.head()

id_props building_status building_tax_expenses       city  \
0  a0I2A00000XQGKnUAP             NaN                   NaN  City-1975   
1  a0I2A00000XQGKtUAP             NaN                   NaN  City-0648   
2  a0I2A00000XQGKwUAP             NaN                   NaN  City-0424   
3  a0I2A00000XQGKxUAP             NaN                   NaN  City-0283   
4  a0I2A00000XQGKzUAP             NaN                   NaN  City-0981   

     class       county sale_status  portfolio       market  num_buildings  \
0  Class A  County-0465           Y       True  Market-0239            NaN   
1  Class A  County-0361           N      False  Market-0076            1.0   
2      NaN  County-0069           N      False  Market-0034            1.0   
3      NaN  County-0404           N      False  Market-0149            NaN   
4  Class B  County-0203           N      False  Market-0112           20.0   

     ...      price_per_unit  property_type_1  region__c  sale_amount__c  \
0    ...          2109090.91    Multi-Housing  Southeast     21440582.78   
1    ...                 NaN    Multi-Housing       West             NaN   
2    ...                 NaN           Retail  Southwest             NaN   
3    ...                 NaN       Industrial  Southeast             NaN   
4    ...           106145.83    Multi-Housing  Southeast     45855000.00   

   sale_date__c size_acres   size_sf  size_units year_built  sale_year  
0    12/31/2016      19.60  508080.0       275.0     2014.0     2016.0  
1           NaN       0.67   62524.0       100.0     2014.0        NaN  
2           NaN        NaN    8640.0         NaN     2012.0        NaN  
3           NaN        NaN  513734.0         NaN     2011.0        NaN  
4    10/31/2016      35.12  363747.0       432.0     1990.0     2016.0  

[5 rows x 24 columns]

#### Deal to Properties: This table comprises information on the deals that has materialized on the properties.Deal and properties mapping

In [13]:
deal_prop.head()

id            id_deals            id_props
0  a0A2A00000TxIrIUAV  0062A00000o979GQAQ  a0I2A00000XQH8YUAX
1  a0A2A00000TxIrSUAV  0062A00000o94JrQAI  a0I2A00000XQJ4dUAH
2  a0A2A00000TxIrXUAV  0062A00000o94GJQAY  a0I2A00000XQJL0UAP
3  a0A2A00000TxIrcUAF  0062A00000o982fQAA  a0I2A00000XQKgkUAH
4  a0A2A00000TxIrhUAF  0062A00000o96M4QAI  a0I2A00000XQMEhUAP

In [462]:
deal_prop.shape
deal_prop['id'].nunique()    ##unique id's
deal_prop['id_deals'].nunique()  ##in single deal more than 1 property sold.
deal_prop['id_props'].nunique()  ##1 property can sold out more than 1 time.that's why count is less
len(set(deal_prop.id_props.unique()).intersection(acc_prop.id_props.unique()))    ##need to find out this relationship??

(25820, 3)

25820

20350

19701

15526

#### Opportunities: These include the historic deals for the accounts. Basically, this gives a transaction summary of the deals that have happened between the accounts and the properties. Succesful deal information

In [55]:
oppor.head(10)

id_accs accounting_date  best_initial_bid            id_deals  \
0  0012A000023XvIOQA0             NaN               NaN  0062A00000moF1LQAU   
1  0012A000023YFe1QAG             NaN               NaN  0062A00000moF9LQAU   
2  0012A000023YEDcQAO       8/12/2004               NaN  0062A00000moGodQAE   
3  0012A000023YA3nQAG             NaN               NaN  0062A00000moBTHQA2   
4  0012A000023XpRUQA0             NaN               NaN  0062A00000mo8OjQAI   
5  0012A000023Y6OGQA0      12/27/2011               NaN  0062A00000moHgpQAE   
6  0012A000023XtJbQAK        6/5/2006               NaN  0062A00000moBqVQAU   
7  0012A000023XtJbQAK        6/5/2006               NaN  0062A00000moCA1QAM   
8  0012A000023XtJbQAK        3/2/2006               NaN  0062A00000moCDiQAM   
9  0012A000023YAtbQAG       7/16/2008               NaN  0062A00000moDLeQAM   

    closedate      date_closed          deal_type  deal_update_flag  \
0   8/12/2005   8/18/2005 0:00         Fixed-rate              True   
1    2/6/2012  6/27/2012 12:00         Fixed-rate              True   
2  11/24/2003   8/11/2004 0:00  Construction Loan              True   
3   2/10/2006   4/19/2006 0:00         Fixed-rate              True   
4   7/13/2006  11/16/2006 0:00         Fixed-rate              True   
5   8/16/2011  12/21/2011 0:00      Floating-Rate              True   
6   1/12/2006   5/30/2006 0:00  Construction Loan              True   
7   1/12/2006   5/30/2006 0:00  Construction Loan              True   
8   1/12/2006   2/28/2006 0:00  Construction Loan              True   
9    5/5/2008   7/16/2008 0:00         Fixed-rate              True   

   debt_yield  fiscal  fiscalquarter  fiscalyear platform property_group  \
0         NaN  2005 3              3        2005     Debt     Healthcare   
1         NaN  2012 1              1        2012     Debt     Healthcare   
2         NaN  2003 4              4        2003     Debt     Healthcare   
3         NaN  2006 1              1        2006     Debt     Healthcare   
4         NaN  2006 3              3        2006     Debt     Healthcare   
5         0.0  2011 3              3        2011     Debt     Healthcare   
6         NaN  2006 1              1        2006     Debt     Healthcare   
7         NaN  2006 1              1        2006     Debt     Healthcare   
8         NaN  2006 1              1        2006     Debt     Healthcare   
9         0.0  2008 2              2        2008     Debt     Healthcare   

     property_type  
0       Healthcare  
1          Biotech  
2          Biotech  
3       Healthcare  
4       Healthcare  
5       Healthcare  
6  Seniors Housing  
7  Seniors Housing  
8  Seniors Housing  
9       Healthcare

In [56]:
oppor.shape
oppor['id_accs'].nunique()   ##need to find out how this have more access id's then which in account table 2756
oppor['id_deals'].nunique()
len(set(deal_prop.id_deals.unique()).intersection(oppor.id_deals.unique()))
len(set(acc_prop.id_accs.unique()).intersection(oppor.id_accs.unique()))
len(set(account.id_accs.unique()).intersection(oppor.id_accs.unique()))

(26029, 15)

7245

26029

14438

1179

1179

#### Property Information.This database contains the universal list of properties and its details

In [360]:
propert.head(10)

id_props building_status  \
0  a0I2A00000XQGKnUAP             NaN   
1  a0I2A00000XQGKtUAP             NaN   
2  a0I2A00000XQGKwUAP             NaN   
3  a0I2A00000XQGKxUAP             NaN   
4  a0I2A00000XQGKzUAP             NaN   
5  a0I2A00000XQGL2UAP        Existing   
6  a0I2A00000XQGL4UAP             NaN   
7  a0I2A00000XQGLCUA5             NaN   
8  a0I2A00000XQGLFUA5        Existing   
9  a0I2A00000XQGLGUA5             NaN   

                      building_tax_expenses       city    class       county  \
0                                       NaN  City-1975  Class A  County-0465   
1                                       NaN  City-0648  Class A  County-0361   
2                                       NaN  City-0424      NaN  County-0069   
3                                       NaN  City-0283      NaN  County-0404   
4                                       NaN  City-0981  Class B  County-0203   
5                       2008 Tax @ $0.60/sf  City-0088      NaN  County-0002   
6                                       NaN  City-1808      NaN  County-0053   
7                                       NaN  City-1462      NaN  County-0478   
8  2011 Tax @ $6.49/sf; 2011 Ops @ $0.87/sf  City-1663  Class B  County-0436   
9                                       NaN  City-1663  Class B  County-0436   

  sale_status  portfolio       market  num_buildings     ...      \
0           Y       True  Market-0239            NaN     ...       
1           N      False  Market-0076            1.0     ...       
2           N      False  Market-0034            1.0     ...       
3           N      False  Market-0149            NaN     ...       
4           N      False  Market-0112           20.0     ...       
5           N      False  Market-0065            NaN     ...       
6           N       True  Market-0204            NaN     ...       
7           N      False  Market-0248            NaN     ...       
8           N       True  Market-0183            1.0     ...       
9           N       True  Market-0183            1.0     ...       

   price_per_sq_ft  price_per_unit  property_type_1  region__c  \
0          1141.55      2109090.91    Multi-Housing  Southeast   
1              NaN             NaN    Multi-Housing       West   
2              NaN             NaN           Retail  Southwest   
3              NaN             NaN       Industrial  Southeast   
4           126.06       106145.83    Multi-Housing  Southeast   
5            52.38        19032.26    Hotel-Lodging       West   
6          1195.17       597163.12    Hotel-Lodging  Southwest   
7           188.83             NaN           Retail  Southeast   
8           551.04             NaN       Industrial       West   
9           127.64             NaN       Industrial       West   

   sale_amount__c sale_date__c size_acres   size_sf size_units  year_built  
0     21440582.78   12/31/2016     19.600  508080.0      275.0      2014.0  
1             NaN          NaN      0.670   62524.0      100.0      2014.0  
2             NaN          NaN        NaN    8640.0        NaN      2012.0  
3             NaN          NaN        NaN  513734.0        NaN      2011.0  
4     45855000.00   10/31/2016     35.120  363747.0      432.0      1990.0  
5      2360000.00    9/14/2010      0.700   45052.0      124.0      2009.0  
6     84200000.00    4/30/2015      1.455   70450.0      141.0      2009.0  
7     19000000.00    8/31/2015        NaN  100621.0        NaN      1999.0  
8     25830000.00     4/7/2005      3.158   46875.0        NaN      1986.0  
9     28750000.00     4/6/2005      8.620  225250.0        NaN      1996.0  

[10 rows x 23 columns]

In [13]:
propert.shape
propert['id_props'].nunique()
len(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))
print('if total properties are 18298 & 14613 have record for bought by a account then unbought properties are: ',18298 - 14613)
len(set(propert.id_props.unique()).intersection(deal_prop.id_props.unique()))

(18298, 24)

18298

14613

if total properties are 18298 & 14613 have record for bought by a account then unbought properties are:  3685


17883

In [278]:
pp=propert[propert.year_built<1960]
pp.shape
propert[(propert.building_status == 'Proposed') | (propert.building_status == 'Demolished') | (propert.building_status == 'Under Construction')].shape
propert.building_status.unique()

(0, 23)

(261, 23)

array([nan, 'Existing', 'Proposed', 'Under Construction', 'Demolished'], dtype=object)

#### This dataset has the accounts for which the properties needs to be suggested. Also note that if a property has already been bought by an account, then it cannot be marketed to another account.

In [119]:
test.head()

id_accs
0  0012A00002AXHJUQA5
1  0012A000023XlWaQAK
2  0012A000023Y4bYQAS
3  0012A000023XlbuQAC
4  0012A000023Xlh9QAC

In [163]:
test.shape
len(set(test.id_accs.unique()).intersection(oppor.id_accs.unique()))
len(set(test.id_accs.unique()).intersection(acc_prop.id_accs.unique()))
## both zero shows no interaction record for the test account ids.seem like new user's for test prediction's

(29, 1)

0

0

In [164]:
##get test id's record from the main Account file.
test_acc = account.loc[account['id_accs'].isin(list(test.id_accs.values))]

In [165]:
test_acc.head(10)

id_accs  active_deals  activity_count  buyer_book  \
4   0012A000023YBFRQA4           283          194195           1   
6   0012A000023XlWaQAK           261          127948           1   
11  0012A000023YDnWQAW           606          217301           1   
17  0012A000023Y9gNQAS           217          200215           1   
20  0012A000023Xo0OQAS           364          157298           1   
22  0012A000023Y4bYQAS           379          204360           1   
32  0012A000023XlkNQAS           288          210435           1   
34  0012A000023XlnqQAC            76          203637           1   
37  0012A000023Xm35QAC           278           27290           1   
40  0012A000023XwMRQA0            47           70500           1   

    servicing_contract  investor_type  cmbs  consultant  correspondent  \
4                    0              8     0           0              0   
6                    0              8     0           0              0   
11                   1              8     0           0              0   
17                   0              1     0           0              0   
20                   0              8     0           0              0   
22                   0              8     0           0              0   
32                   0              1     0           0              0   
34                   0              1     0           0              0   
37                   1              8     0           0              1   
40                   0              1     0           0              0   

    foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
4         0                0            1                1               0   
6         0                0            1                1               0   
11        0                0            1                1               0   
17        0                0            1                1               0   
20        0                0            1                1               0   
22        0                0            1                1               0   
32        0                0            1                1               0   
34        1                0            1                1               0   
37        0                0            1                1               0   
40        0                0            1                1               0   

    num_deals_as_client  num_deals_as_investor  number_of_properties  \
4                    63                    220                   393   
6                    40                    221                   138   
11                   51                    555                   108   
17                  170                     47                   558   
20                  126                    238                    33   
22                   92                    287                    86   
32                  224                     64                   247   
34                   35                     41                   148   
37                   25                    253                     7   
40                   32                     15                   204   

    number_of_related_deals  number_of_related_properties  \
4                       383                           591   
6                       283                           276   
11                      664                           381   
17                      339                           643   
20                      410                           164   
22                      478                           359   
32                      355                           299   
34                      128                           193   
37                      189                            73   
40                       54                           229   

    number_of_won_deals_as_client  
4                              46  
6                              30  
11                 

## Findings:
1. Inconsistent number of records are in all the main and interaction table.need to take only those interaction which have in main Account and property table.delete others
2. Content based can't be used because user's in test are new.so,only collaborative filtering and hybrid can be applied.

## SVD

In [27]:
acc_prop['id_props'].nunique()

63001

In [61]:
intre = acc_prop.loc[acc_prop['id_props'].isin(list(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique())))]
acc_prop.loc[acc_prop['id_props'].isin(list(set(propert.id_props.unique()).intersection(deal_prop.id_props.unique())))].shape

(11450, 3)

In [76]:
intre.shape
intre.head()

(11450, 4)

id             id_accs            id_props  rate
1   a062A00001VmP7EQAV  0012A000023XmOxQAK  a0I2A00000XvUEwUAN     1
86  a062A00001VluvmQAB  0012A000023YLqwQAG  a0I2A00000XQIHiUAP     1
88  a062A00001VlwF1QAJ  0012A000023YLqwQAG  a0I2A00000XwOPFUA3     1
93  a062A00001Vm40OQAR  0012A000023YLqwQAG  a0I2A00000XQJwiUAH     1
97  a062A00001Vm7RhQAJ  0012A000023YLqwQAG  a0I2A00000XQGhNUAX     1

In [63]:
len(list(set(propert.id_props.unique()).intersection(intre.id_props.unique())))

8121

In [64]:
intre['rate']=1

C:\Users\royal\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
intre_r = intre.drop('id',axis=1)
# propert.drop('id_deals', axis=1, inplace=True)

In [66]:
intre_r1 = intre_r.sample(frac=1).drop_duplicates(subset='id_props')

In [68]:
intre_m = intre_r1.pivot(index = 'id_accs', columns ='id_props', values = 'rate').fillna(0)

In [69]:
intre_r1.shape
intre_m.shape

(8121, 3)

(1568, 8121)

In [70]:
intre_m

id_props            a0I2A00000XQGKqUAP  a0I2A00000XQGKtUAP  \
id_accs                                                      
0012A000023XlNjQAK                 0.0                 0.0   
0012A000023XlSVQA0                 0.0                 0.0   
0012A000023XlStQAK                 0.0                 0.0   
0012A000023XlUGQA0                 0.0                 0.0   
0012A000023XlVSQA0                 0.0                 0.0   
0012A000023XlVXQA0                 0.0                 0.0   
0012A000023XlVhQAK                 0.0                 0.0   
0012A000023XlVkQAK                 0.0                 0.0   
0012A000023XlVrQAK                 0.0                 0.0   
0012A000023XlVwQAK                 0.0                 0.0   
0012A000023XlWBQA0                 0.0                 0.0   
0012A000023XlWMQA0                 0.0                 0.0   
0012A000023XlWvQAK                 0.0                 0.0   
0012A000023XlX9QAK                 0.0                 0.0   
0012A000023XlXsQAK                 0.0                 0.0   
0012A000023XlYMQA0                 0.0                 0.0   
0012A000023XlYRQA0                 0.0                 0.0   
0012A000023XlYgQAK                 0.0                 0.0   
0012A000023XlYlQAK                 0.0                 0.0   
0012A000023XlYvQAK                 0.0                 0.0   
0012A000023XlZ5QAK                 0.0                 0.0   
0012A000023XlZFQA0                 0.0                 0.0   
0012A000023XlZPQA0                 0.0                 0.0   
0012A000023XlZUQA0                 0.0                 0.0   
0012A000023XlZyQAK                 0.0                 0.0   
0012A000023XlaOQAS                 0.0                 0.0   
0012A000023XlbGQAS                 0.0                 0.0   
0012A000023XlbHQAS                 0.0                 0.0   
0012A000023XlbLQAS                 0.0                 0.0   
0012A000023XlbQQAS                 0.0                 0.0   
...                                ...                 ...   
0012A000029wHvgQAE                 0.0                 0.0   
0012A000029wHw0QAE                 0.0                 0.0   
0012A000029wI1XQAU                 0.0                 0.0   
0012A000029wI3IQAU                 0.0                 0.0   
0012A000029wI3eQAE                 0.0                 0.0   
0012A000029wIAWQA2                 0.0                 0.0   
0012A000029wIHnQAM                 0.0                 0.0   
0012A000029wIJjQAM                 0.0                 0.0   
0012A000029wILTQA2                 0.0                 0.0   
0012A000029wIO5QAM                 0.0                 0.0   
0012A000029wIP5QAM                 0.0                 0.0   
0012A00002AMThyQAH                 0.0                 0.0   
0012A00002AMTiaQAH                 0.0                 0.0   
0012A00002AMTqFQAX                 0.0                 0.0   
0012A00002AOBifQAH                 0.0                 0.0   
0012A00002An3i4QAB                 0.0                 0.0   
0012A00002An3pdQAB                 0.0                 0.0   
0012A00002BFHHLQA5                 0.0                 0.0   
0012A00002BFKIxQAP                 0.0                 0.0   
0012A00002Bg1kGQAR                 0.0                 0.0   
0012A00002Bg1wEQAR                 0.0                 0.0   
0012A00002Bg1z6QAB                 0.0                 0.0   
0012A00002Bg23PQAR                 0.0                 0.0   
0012A00002Bg263QAB                 0.0                 0.0   
0012A00002Bg2LxQAJ                 0.0                 0.0   
0012A00002Do4r9QAB                 0.0                 0.0   
0012A00002Dp3dLQAR                 0.0                 0.0   
0012A00002HqXOHQA3                 0.0                 0.0   
0012A00002IbcsiQAB                 0.0                 0.0   
001F000001u2YKzIAM                 0.0                 0.0   

id_props            a0I2A00000XQGKvUAP  a0I2A00000XQGKwUAP  \
id_accs                        

In [71]:
R = intre_m.as_matrix()
intre_mean = np.mean(R, axis = 1)
intre_demeaned = R - intre_mean.reshape(-1, 1)

In [72]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(intre_demeaned, k = 50)

In [73]:
sigma = np.diag(sigma)
all_predicted_user  = np.dot(np.dot(U, sigma), Vt) + intre_mean.reshape(-1, 1)

In [74]:
preds = pd.DataFrame(all_predicted_user, columns = intre_m.columns)

In [75]:
preds.head()

id_props  a0I2A00000XQGKqUAP  a0I2A00000XQGKtUAP  a0I2A00000XQGKvUAP  \
0                   0.000017            0.001522            0.000450   
1                   0.000007            0.000206            0.000090   
2                   0.000006            0.002003            0.000464   
3                   0.000007            0.000206            0.000090   
4                  -0.000060           -0.001760           -0.001076   

id_props  a0I2A00000XQGKwUAP  a0I2A00000XQGKxUAP  a0I2A00000XQGKzUAP  \
0                   0.001332            0.000147            0.000010   
1                   0.000184            0.000042            0.000004   
2                   0.001737            0.000110            0.000004   
3                   0.000184            0.000042            0.000004   
4                   0.001173           -0.000415           -0.000038   

id_props  a0I2A00000XQGL2UAP  a0I2A00000XQGL4UAP  a0I2A00000XQGL7UAP  \
0                   0.001308            0.001349            0.001406   
1                   0.000181            0.000186            0.000192   
2                   0.001705            0.001762            0.001840   
3                   0.000181            0.000186            0.000192   
4                   0.001707            0.000795           -0.000241   

id_props  a0I2A00000XQGLCUA5         ...          a0I2A00000ZGs1TUAT  \
0                   0.001308         ...                    0.000208   
1                   0.000181         ...                    0.000053   
2                   0.001705         ...                    0.000173   
3                   0.000181         ...                    0.000053   
4                   0.001707         ...                   -0.000555   

id_props  a0I2A00000ZGs1UUAT  a0I2A00000ZGsGoUAL  a0I2A00000ZGtDiUAL  \
0                   0.000208            0.001359            0.000219   
1                   0.000053            0.000187            0.000055   
2                   0.000173            0.001775            0.000185   
3                   0.000053            0.000187            0.000055   
4                  -0.000555            0.000599           -0.000580   

id_props  a0I2A00000ZGtDxUAL  a0I2A00000ZGtE3UAL  a0I2A00000ZGu3uUAD  \
0                   0.001332            0.001332            0.000046   
1                   0.000184            0.000184            0.000017   
2                   0.001737            0.001737            0.000024   
3                   0.000184            0.000184            0.000017   
4                   0.001173            0.001173           -0.000154   

id_props  a0I2A00000ZGugoUAD  a0I2A00000ZGvMpUAL  a0I2A00000ZGwYxUAL  
0                   0.001349            0.001340            0.000188  
1                   0.000186            0.000185            0.000050  
2                   0.001762            0.001749            0.000152  
3                   0.000186            0.000185            0.000050  
4                   0.000795            0.000986           -0.000512  

[5 rows x 8121 columns]

In [77]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD, evaluate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(intre[['id_accs', 'id_props', 'rate']], reader)

# Split the dataset for 5-fold evaluation
data.split(n_folds=5)

In [78]:
# Use the SVD algorithm.
svd = SVD()
# Compute the RMSE of the SVD algorithm.
evaluate(svd, data, measures=['RMSE'])

C:\Users\royal\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\royal\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.0409
------------
Fold 2
RMSE: 0.0432
------------
Fold 3
RMSE: 0.0434
------------
Fold 4
RMSE: 0.0407
------------
Fold 5
RMSE: 0.0431
------------
------------
Mean RMSE: 0.0423
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.040923891638818995,
                             0.043225331027440958,
                             0.043421733669020773,
                             0.040702741660900071,
                             0.043059479558149587]})

In [79]:
trainset = data.build_full_trainset()
svd.train(trainset)

C:\Users\royal\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [80]:
intre[intre.id_accs =='0012A000023XmOxQAK']

id             id_accs            id_props  rate
1  a062A00001VmP7EQAV  0012A000023XmOxQAK  a0I2A00000XvUEwUAN     1

In [84]:
svd.predict('0012A000023XmOxQAK', 'a0I2A00000XwOPFUA3')

Prediction(uid='0012A000023XmOxQAK', iid='a0I2A00000XwOPFUA3', r_ui=None, est=1, details={'was_impossible': False})

### Process Account File

In [166]:
account.head()

id_accs  active_deals  activity_count  buyer_book  \
0  0012A000023XlCCQA0             1            2367           1   
1  0012A000023XmOxQAK             9            7974           1   
2  0012A000023YLqwQAG           891           50161           1   
3  0012A000023XlNjQAK            34           14631           1   
4  0012A000023YBFRQA4           283          194195           1   

   servicing_contract  investor_type  cmbs  consultant  correspondent  \
0                   0             10     0           0              0   
1                   0             13     0           0              0   
2                   1              3     0           0              0   
3                   0              4     1           0              0   
4                   0              8     0           0              0   

   foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
0        0                0            0                0               0   
1        0                0            1                1               0   
2        0                0            1                1               0   
3        1                0            1                1               0   
4        0                0            1                1               0   

   num_deals_as_client  num_deals_as_investor  number_of_properties  \
0                    0                      1                     1   
1                    7                      2                    77   
2                   40                    851                   693   
3                    0                     34                     0   
4                   63                    220                   393   

   number_of_related_deals  number_of_related_properties  \
0                        2                             1   
1                       18                            85   
2                      996                          1167   
3                       45                            14   
4                      383                           591   

   number_of_won_deals_as_client  
0                              0  
1                              7  
2                             37  
3                              0  
4                             46

In [167]:
account['investor_type'].fillna('Unknown',inplace=True)

In [168]:
##label encode the string data
ls = ['buyer_book','servicing_contract','investor_type','cmbs' ,'consultant' ,'correspondent' ,'foreign' ,'master_servicer' ,'lender_book' ,'loan_sales_book' ,'loan_servicing']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in ls:
    le = preprocessing.LabelEncoder()
    account[i] = le.fit_transform(account[i])

### Knn on account file

In [184]:
##KNN to get 5 nearest neighbours
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(account.iloc[:,1:], account['id_accs'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [183]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(account.iloc[:,1:])

In [91]:
distances, indices = nbrs.kneighbors(account.iloc[:,1:])

In [117]:
# test_acc.reset_index(inplace = True)
test_acc.head()

index             id_accs  active_deals  activity_count  buyer_book  \
0      4  0012A000023YBFRQA4           283          194195        True   
1      6  0012A000023XlWaQAK           261          127948        True   
2     11  0012A000023YDnWQAW           606          217301        True   
3     17  0012A000023Y9gNQAS           217          200215        True   
4     20  0012A000023Xo0OQAS           364          157298        True   

   servicing_contract           investor_type   cmbs  consultant  \
0               False  Life Insurance Company  False       False   
1               False  Life Insurance Company  False       False   
2                True  Life Insurance Company  False       False   
3               False     Advisor/Equity Fund  False       False   
4               False  Life Insurance Company  False       False   

   correspondent              ...                master_servicer  lender_book  \
0          False              ...                          False         True   
1          False              ...                          False         True   
2          False              ...                          False         True   
3          False              ...                          False         True   
4          False              ...                          False         True   

   loan_sales_book  loan_servicing  num_deals_as_client  \
0             True           False                   63   
1             True           False                   40   
2             True           False                   51   
3             True           False                  170   
4             True           False                  126   

   num_deals_as_investor  number_of_properties  number_of_related_deals  \
0                    220                   393                      383   
1                    221                   138                      283   
2                    555                   108                      664   
3                     47                   558                      339   
4                    238                    33                      410   

   number_of_related_properties  number_of_won_deals_as_client  
0                           591                             46  
1                           276                             30  
2                           381                             38  
3                           643                            133  
4                           164                            103  

[5 rows x 21 columns]

In [181]:
dd={}
for i,j in zip(test_acc.index.values,test_acc.id_accs.values):
    print(i,j)
    indices[i:i+1]
    test_id = indices[i:i+1][0][0]
    map_id = indices[i:i+1][0][1]
    distances[i:i+1]
    account.iloc[test_id:test_id+1,:]
    account.iloc[map_id:map_id+1,:]
    map_prop_id = account.iloc[map_id:map_id+1,:].id_accs.values
    map_prop_id[0]
#     propert[propert.id_props.isin(deal_prop[deal_prop.id_deals.isin(oppor[oppor.id_accs ==map_prop_id[0]].id_deals)].id_props)].head(1)
    propert[propert.id_props.isin(acc_prop[acc_prop.id_accs == map_prop_id[0]].id_props)].head(1)
    dd[j]=propert[propert.id_props.isin(acc_prop[acc_prop.id_accs == map_prop_id[0]].id_props)].id_props.values
    print('###################')

4 0012A000023YBFRQA4


array([[  4, 101]], dtype=int64)

array([[    0.        ,  1227.82612776]])

id_accs  active_deals  activity_count  buyer_book  \
4  0012A000023YBFRQA4           283          194195           1   

   servicing_contract  investor_type  cmbs  consultant  correspondent  \
4                   0              8     0           0              0   

   foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
4        0                0            1                1               0   

   num_deals_as_client  num_deals_as_investor  number_of_properties  \
4                   63                    220                   393   

   number_of_related_deals  number_of_related_properties  \
4                      383                           591   

   number_of_won_deals_as_client  
4                             46

id_accs  active_deals  activity_count  buyer_book  \
101  0012A000023XowrQAC           108          193100           1   

     servicing_contract  investor_type  cmbs  consultant  correspondent  \
101                   0             10     0           0              0   

     foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
101        0                0            1                1               0   

     num_deals_as_client  num_deals_as_investor  number_of_properties  \
101                   94                     14                   163   

     number_of_related_deals  number_of_related_properties  \
101                      224                           197   

     number_of_won_deals_as_client  
101                             79

...

id_accs  active_deals  activity_count  buyer_book  \
907  0012A00002JF3YmQAL            68          113036           1   

     servicing_contract  investor_type  cmbs  consultant  correspondent  \
907                   0              1     0           0              0   

     foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
907        0                0            1                1               0   

     num_deals_as_client  num_deals_as_investor  number_of_properties  \
907                   40                     28                    12   

     number_of_related_deals  number_of_related_properties  \
907                        3                            33   

     number_of_won_deals_as_client  
907                             10

id_accs  active_deals  activity_count  buyer_book  \
159  0012A000023Xly3QAC            65          116097           1   

     servicing_contract  investor_type  cmbs  consultant  correspondent  \
159                   0             10     0           0              0   

     foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
159        1                0            1                1               0   

     num_deals_as_client  num_deals_as_investor  number_of_properties  \
159                   45                     20                    43   

     number_of_related_deals  number_of_related_properties  \
159                      133                            86   

     number_of_won_deals_as_client  
159                             34

'0012A000023Xly3QAC'

Empty DataFrame
Columns: [id_props, building_status, building_tax_expenses, city, class, county, sale_status, portfolio, market, num_buildings, num_floors, num_parking_spaces, occupancy_rate, price_per_sq_ft, price_per_unit, property_type_1, region__c, sale_amount__c, sale_date__c, size_acres, size_sf, size_units, year_built, sale_year]
Index: []

[0 rows x 24 columns]

###################


In [126]:
indices[11:21]
distances[11:21]

array([[  11,   32],
       [  12, 1076],
       [  13,  412],
       [  14, 2533],
       [  15, 1255],
       [  16,   39],
       [  17,   79],
       [  18,  660],
       [  19, 1732],
       [  20,  105]], dtype=int64)

array([[    0.        ,  6903.48940754],
       [    0.        ,    46.11941023],
       [    0.        ,    40.17461885],
       [    0.        ,    11.91637529],
       [    0.        ,    90.59801322],
       [    0.        ,   367.7662301 ],
       [    0.        ,  3177.78806719],
       [    0.        ,   201.57380782],
       [    0.        ,   152.79070652],
       [    0.        ,   626.01677294]])

In [182]:
dd

{'0012A000023XlWaQAK': array([], dtype=object),
 '0012A000023XlbuQAC': array([], dtype=object),
 '0012A000023Xlh9QAC': array([], dtype=object),
 '0012A000023XlhdQAC': array([], dtype=object),
 '0012A000023XlkNQAS': array([], dtype=object),
 '0012A000023XlnWQAS': array([], dtype=object),
 '0012A000023XlnqQAC': array([], dtype=object),
 '0012A000023Xm35QAC': array([], dtype=object),
 '0012A000023XmRwQAK': array([], dtype=object),
 '0012A000023XmmFQAS': array([], dtype=object),
 '0012A000023Xo0OQAS': array([], dtype=object),
 '0012A000023XoRjQAK': array([], dtype=object),
 '0012A000023Xp4LQAS': array([], dtype=object),
 '0012A000023Xp5LQAS': array([], dtype=object),
 '0012A000023XrTNQA0': array([], dtype=object),
 '0012A000023XwMRQA0': array([], dtype=object),
 '0012A000023Y1rtQAC': array([], dtype=object),
 '0012A000023Y4bYQAS': array([], dtype=object),
 '0012A000023Y9gNQAS': array([], dtype=object),
 '0012A000023YAupQAG': array([], dtype=object),
 '0012A000023YB67QAG': array([], dtype=o

In [115]:
account.iloc[20:22,:]
account.iloc[105:107,:]

id_accs  active_deals  activity_count  buyer_book  \
20  0012A000023Xo0OQAS           364          157298           1   
21  0012A000023XnT9QAK            20            8031           1   

    servicing_contract  investor_type  cmbs  consultant  correspondent  \
20                   0              8     0           0              0   
21                   0              2     0           0              0   

    foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
20        0                0            1                1               0   
21        0                0            1                1               0   

    num_deals_as_client  num_deals_as_investor  number_of_properties  \
20                  126                    238                    33   
21                    0                     20                    56   

    number_of_related_deals  number_of_related_properties  \
20                      410                           164   
21                       25                           423   

    number_of_won_deals_as_client  
20                            103  
21                              0

id_accs  active_deals  activity_count  buyer_book  \
105  0012A000023XleaQAC            72          157694           1   
106  0012A000023Xo4PQAS             9           46895           1   

     servicing_contract  investor_type  cmbs  consultant  correspondent  \
105                   0              1     0           0              0   
106                   0              1     0           0              0   

     foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
105        0                0            1                1               0   
106        1                0            1                1               0   

     num_deals_as_client  num_deals_as_investor  number_of_properties  \
105                   39                     33                   128   
106                    5                      4                    30   

     number_of_related_deals  number_of_related_properties  \
105                      117                           138   
106                       17                            33   

     number_of_won_deals_as_client  
105                             35  
106                              5

In [116]:
acc_prop[acc_prop.id_accs == 'a0I2A00000XQLImUAP']
propert[propert.id_props.isin(acc_prop[acc_prop.id_accs =='a0I2A00000XQLImUAP'].id_props)].head()
propert[propert.id_props.isin(deal_prop[deal_prop.id_deals.isin(oppor[oppor.id_accs =='0012A000023XleaQAC'].id_deals)].id_props)].head()

id             id_accs            id_props
7664  a062A00001VluwHQAR  0012A000023XleaQAC  a0I2A00000XQKtbUAH
7665  a062A00001VmP7JQAV  0012A000023XleaQAC  a0I2A00000XQNHZUA5
7666  a062A00001VmPxbQAF  0012A000023XleaQAC  a0I2A00000XuQKYUA3
7667  a062A00001Vmq1WQAR  0012A000023XleaQAC  a0I2A00000XQKtoUAH
7668  a062A00001VmtkNQAR  0012A000023XleaQAC  a0I2A00000XuNwYUAV
7669  a062A00001Vmuz4QAB  0012A000023XleaQAC  a0I2A00000XwMJHUA3
7670  a062A00001Vmv3VQAR  0012A000023XleaQAC  a0I2A00000XwPqfUAF
7671  a062A00001WhjW8QAJ  0012A000023XleaQAC  a0I2A00000XQGLiUAP
7672  a062A00001WhjbnQAB  0012A000023XleaQAC  a0I2A00000XQGOuUAP
7673  a062A00001WhkDbQAJ  0012A000023XleaQAC  a0I2A00000XQGncUAH
7674  a062A00001WhkHoQAJ  0012A000023XleaQAC  a0I2A00000XQGsJUAX
7675  a062A00001WhkJlQAJ  0012A000023XleaQAC  a0I2A00000XQGtqUAH
7676  a062A00001WhkYVQAZ  0012A000023XleaQAC  a0I2A00000XQH3yUAH
7677  a062A00001WhkZ1QAJ  0012A000023XleaQAC  a0I2A00000XQH2DUAX
7678  a062A00001WhkuCQAR  0012A000023XleaQAC  a0I2A00000XQHLEUA5
7679  a062A00001Whl1uQAB  0012A000023XleaQAC  a0I2A00000XQHOSUA5
7680  a062A00001WhlDJQAZ  0012A000023XleaQAC  a0I2A00000XQHX7UAP
7681  a062A00001WhlOrQAJ  0012A000023XleaQAC  a0I2A00000XQHg0UAH
7682  a062A00001WhlS9QAJ  0012A000023XleaQAC  a0I2A00000XQHiPUAX
7683  a062A00001WhltBQAR  0012A000023XleaQAC  a0I2A00000XQI39UAH
7684  a062A00001WhlyqQAB  0012A000023XleaQAC  a0I2A00000XQI6lUAH
7685  a062A00001WhmCpQAJ  0012A000023XleaQAC  a0I2A00000XQIGoUAP
7686  a062A00001WhmN8QAJ  0012A000023XleaQAC  a0I2A00000XQIPeUAP
7687  a062A00001Whmb0QAB  0012A000023XleaQAC  a0I2A00000XQIYfUAP
7688  a062A00001WhmbbQAB  0012A000023XleaQAC  a0I2A00000XQIZ3UAP
7689  a062A00001WhmcuQAB  0012A000023XleaQAC  a0I2A00000XQIZpUAP
7690  a062A00001WhmmKQAR  0012A000023XleaQAC  a0I2A00000XQIhhUAH
7691  a062A00001WhnCaQAJ  0012A000023XleaQAC  a0I2A00000XQJ0ZUAX
7692  a062A00001WhnL9QAJ  0012A000023XleaQAC  a0I2A00000XQJ6dUAH
7693  a062A00001WhnQ2QAJ  0012A000023XleaQAC  a0I2A00000XQJEbUAP
...                  ...                 ...                 ...
7772  a062A00001WjFBwQAN  0012A000023XleaQAC  a0I2A00000XuRUoUAN
7773  a062A00001Yq0qtQAB  0012A000023XleaQAC  a0I2A00000XwiNQUAZ
7774  a062A00001Yq0wdQAB  0012A000023XleaQAC  a0I2A00000XwiQFUAZ
7775  a062A00001YqSwYQAV  0012A000023XleaQAC  a0I2A00000XxRDeUAN
7776  a062A00001YruUZQAZ  0012A000023XleaQAC  a0I2A00000YFhTkUAL
7777  a062A00001YsG6jQAF  0012A000023XleaQAC  a0I2A00000YG2gXUAT
7778  a062A00001YsGBTQA3  0012A000023XleaQAC  a0I2A00000YG2m5UAD
7779  a062A00001YshYZQAZ  0012A000023XleaQAC  a0I2A00000YGXdlUAH
7780  a062A00001YsiUHQAZ  0012A000023XleaQAC  a0I2A00000YGYiBUAX
7781  a062A00001Ysj9gQAB  0012A000023XleaQAC  a0I2A00000YGZWuUAP
7782  a062A00001Ysjn3QAB  0012A000023XleaQAC  a0I2A00000YGaLiUAL
7783  a062A00001YskLIQAZ  0012A000023XleaQAC  a0I2A00000YGb5uUAD
7784  a062A00001YskpUQAR  0012A000023XleaQAC  a0I2A00000YGbiYUAT
7785  a062A00001YskrzQAB  0012A000023XleaQAC  a0I2A00000YGblrUAD
7786  a062A00001YslK5QAJ  0012A000023XleaQAC  a0I2A00000YGcLxUAL
7787  a062A00001YslMTQAZ  0012A000023XleaQAC  a0I2A00000YGcOwUAL
7788  a062A00001YsuarQAB  0012A000023XleaQAC  a0I2A00000YGnWeUAL
7789  a062A00001Yswn1QAB  0012A000023XleaQAC  a0I2A00000YGprZUAT
7790  a062A00001Yt2MaQAJ  0012A000023XleaQAC  a0I2A00000YGv8DUAT
7791  a062A00001YuHD6QAN  0012A000023XleaQAC  a0I2A00000YI8OpUAL
7792  a062A00001YuOfCQAV  0012A000023XleaQAC  a0I2A00000YIGE0UAP
7793  a062A00001YuOiMQAV  0012A000023XleaQAC  a0I2A00000YIGHRUA5
7794  a062A00001YuQtEQAV  0012A000023XleaQAC  a0I2A00000YIIbvUAH
7795  a062A00001YuQwqQAF  0012A000023XleaQAC  a0I2A00000YIIfQUAX
7796  a062A00001YuQxUQAV  0012A000023XleaQAC  a0I2A00000YIIgGUAX
7797  a062A00001YuQyCQAV  0012A000023XleaQAC  a0I2A00000YIIh9UAH
7798  a062A00001Yum0rQAB  0012A000023XleaQAC  a0I2A00000YIel5UAD
7799  a062A00001YumOhQAJ  0012A000023XleaQAC  a0I2A00000YIf9jUAD
7800  a062A00001YvAniQAF  0012A000023XleaQAC  a0I2A00000XuQ3wUAF
7801  a062A00001YvAr1QAF  0012A000023XleaQAC  a0I2A00000Y

id_props building_status  \
126   a0I2A00000XQGOuUAP        Existing   
734   a0I2A00000XQGncUAH        Existing   
864   a0I2A00000XQGsJUAX        Existing   
908   a0I2A00000XQGtqUAH        Existing   
1109  a0I2A00000XQH2DUAX        Existing   

                                  building_tax_expenses       city    class  \
126                            2014 Tax @ $5059.11/Unit  City-1195  Class A   
734                              2010 Tax @ $27.65/Unit  City-0090  Class A   
864   2016 Tax @ $1.73/sf, 2012 Est Tax @ $1.94/sf; ...  City-0032      NaN   
908                            2016 Tax @ $2110.46/Unit  City-0970  Class B   
1109                                2012 Tax @ $0.42/sf  City-1813  Class A   

           county sale_status  portfolio       market  num_buildings  \
126   County-0385           N      False  Market-0140            1.0   
734   County-0636           N      False  Market-0016            5.0   
864   County-0239           N       True  Market-0013            NaN   
908   County-0172           N      False  Market-0059           19.0   
1109  County-0562           N       True  Market-0205            1.0   

        ...      price_per_unit          property_type_1  region__c  \
126     ...           431506.85            Multi-Housing       West   
734     ...           163437.21            Multi-Housing  Southwest   
864     ...                 NaN                   Retail  Southeast   
908     ...           122797.36            Multi-Housing  Southwest   
1109    ...                 NaN  Medical Office Building       West   

      sale_amount__c  sale_date__c size_acres   size_sf  size_units  \
126       63000000.0     4/10/2015     1.4793  119164.0       146.0   
734       71585500.0      5/3/2012    10.0000  409076.0       438.0   
864       43150000.0      9/1/2011    13.3600  128727.0         NaN   
908      111500000.0     12/4/2014    17.6200  792786.0       908.0   
1109     146100000.0     1/31/2012     3.0600  150801.0         NaN   

     year_built  sale_year  
126      2010.0     2015.0  
734      2009.0     2012.0  
864      1992.0     2011.0  
908      1996.0     2014.0  
1109     2008.0     2012.0  

[5 rows x 24 columns]

id_props building_status  \
864   a0I2A00000XQGsJUAX        Existing   
1646  a0I2A00000XQHOSUA5        Existing   
2512  a0I2A00000XQI0HUAX        Existing   
2524  a0I2A00000XQI0sUAH        Existing   
2661  a0I2A00000XQI6lUAH        Existing   

                                  building_tax_expenses       city    class  \
864   2016 Tax @ $1.73/sf, 2012 Est Tax @ $1.94/sf; ...  City-0032      NaN   
1646                                2011 Tax @ $2.95/sf  City-0376      NaN   
2512  2016 Tax @ $4.24/sf, 2011 Est Tax @ $3.15/sf; ...  City-1869  Class A   
2524                           2016 Tax @ $1616.24/Unit  City-0329  Class B   
2661                                2016 Tax @ $2.37/sf  City-1618      NaN   

           county sale_status  portfolio       market  num_buildings  \
864   County-0239           N       True  Market-0013            NaN   
1646  County-0160           N      False  Market-0047            3.0   
2512  County-0343           N      False  Market-0214            1.0   
2524  County-0585           N      False  Market-0171           17.0   
2661  County-0010           N      False  Market-0179            3.0   

        ...      price_per_unit  property_type_1     region__c  \
864     ...                 NaN           Retail     Southeast   
1646    ...                 NaN           Retail       Midwest   
2512    ...                 NaN           Office          West   
2524    ...          1176470.59    Multi-Housing     Southeast   
2661    ...                 NaN           Retail  Mid-Atlantic   

      sale_amount__c  sale_date__c size_acres   size_sf  size_units  \
864       43150000.0      9/1/2011      13.36  128727.0         NaN   
1646     130000000.0     4/29/2016       0.00  222851.0         0.0   
2512     368600000.0     7/31/2013       1.96  187373.0         NaN   
2524      40000000.0     2/21/2014      34.00  434608.0        34.0   
2661      58800000.0     1/15/2016       2.00  137974.0         NaN   

     year_built  sale_year  
864      1992.0     2011.0  
1646     2009.0     2016.0  
2512     2001.0     2013.0  
2524     1990.0     2014.0  
2661     2002.0     2016.0  

[5 rows x 24 columns]

In [185]:
##set the index value as row for test to get from accounts table.
test_acc.reset_index(inplace = True)
index = test_acc['index']
index  = index.values

In [186]:
##get top 5 neighbours
top5ids = neigh.kneighbors(account.iloc[index,1:])[1]
print(neigh.kneighbors(account.iloc[index,1:])[1])

[[   4  101  188   17   79]
 [   6 1074  510  480  200]
 [  11   32   22   34   79]
 [  17   79   34   22    4]
 [  20  105   57   58   82]
 [  22   34   79   17   32]
 [  32   22   34   11   79]
 [  34   79   22   17   32]
 [  37  140  901  158  951]
 [  40   13  412  175 2123]
 [  51  148  921   80   93]
 [  54   85  481  350  121]
 [  79   34   22   17   32]
 [  80   93  921  148  109]
 [  91  451   51  148  134]
 [  93   80  921  148  109]
 [ 120   96  135  128  222]
 [ 133  220  227  242  510]
 [ 165  175  271  112 1410]
 [ 200  480 1074    6  510]
 [ 227  220  242  133   86]
 [ 266  511  477  476  739]
 [ 283  450 1151  802  803]
 [ 403  498 2389 1249  314]
 [ 510    6  133 1074  480]
 [ 563 1451  562  263 1004]
 [ 730  494 1550 1000 2037]
 [ 737 1451  563  562  263]
 [ 907  159  185   86   96]]


In [187]:
##index and id_accs dictionary 
dic_acc = {}
k = 0
all_account = account['id_accs']
for i in all_account:
    dic_acc[k] = i
    k = k+1

In [188]:
##get the test id's as key and 5 neighbour's id's as value.
dd={}
fir = list(top5ids[:,0:1].reshape(-1))
sec = list(top5ids[:,1:2].reshape(-1))
thr  =list(top5ids[:,2:3].reshape(-1))
four  =list(top5ids[:,3:4].reshape(-1))
fif = list(top5ids[:,4:5].reshape(-1))
for i in range(29):
    #accounts['id_accs'].loc[test_acc['index'].isin([4])]
    dd[dic_acc[fir[i]]] = [dic_acc[sec[i]], dic_acc[thr[i]], dic_acc[four[i]], dic_acc[fif[i]]]

In [189]:
dd

{'0012A000023XlWaQAK': ['0012A000023XpKsQAK',
  '0012A000023XmRwQAK',
  '0012A000023XonpQAC',
  '0012A000023XlbuQAC'],
 '0012A000023XlbuQAC': ['0012A000023XonpQAC',
  '0012A000023XpKsQAK',
  '0012A000023XlWaQAK',
  '0012A000023XmRwQAK'],
 '0012A000023Xlh9QAC': ['0012A000023XsHdQAK',
  '0012A000023XvtGQAS',
  '0012A000023Xt67QAC',
  '0012A000023XsS6QAK'],
 '0012A000023XlhdQAC': ['0012A000023Xlp3QAC',
  '0012A000023Xpn2QAC',
  '0012A000023YB67QAG',
  '0012A000023XljZQAS'],
 '0012A000023XlkNQAS': ['0012A000023Y4bYQAS',
  '0012A000023XlnqQAC',
  '0012A000023YDnWQAW',
  '0012A00002AXHJUQA5'],
 '0012A000023XlnWQAS': ['0012A000023XrTNQA0',
  '0012A000029wHvgQAE',
  '0012A000023XvdPQAS',
  '0012A000023XoHnQAK'],
 '0012A000023XlnqQAC': ['0012A00002AXHJUQA5',
  '0012A000023Y4bYQAS',
  '0012A000023Y9gNQAS',
  '0012A000023XlkNQAS'],
 '0012A000023Xm35QAC': ['0012A000023Xmd9QAC',
  '0012A000023XmgWQAS',
  '0012A000023Xm28QAC',
  '0012A000023XmDGQA0'],
 '0012A000023XmRwQAK': ['0012A000023XlWaQAK',
  

In [327]:
##make copy of the org. dictionary
ds = dd.copy()

In [328]:
##get dictionary of test id & property id which are available in property file & get from knn.
rm_unk = acc_prop.loc[acc_prop['id_props'].isin(list(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique())))]
for i,j in ds.items():
    print(i,j)
    x = rm_unk.loc[rm_unk['id_accs'].isin(j)]
    if x.shape[0]>0:
        ds[i] = x.groupby('id_accs').apply(lambda df: df.sample(1))['id_props'].values
    else:
        ds[i] = []

0012A000023YBFRQA4 ['0012A000023XowrQAC', '0012A000023XmoVQAS', '0012A000023Y9gNQAS', '0012A00002AXHJUQA5']
0012A000023XlWaQAK ['0012A000023XpKsQAK', '0012A000023XmRwQAK', '0012A000023XonpQAC', '0012A000023XlbuQAC']
0012A000023YDnWQAW ['0012A000023XlkNQAS', '0012A000023Y4bYQAS', '0012A000023XlnqQAC', '0012A00002AXHJUQA5']
0012A000023Y9gNQAS ['0012A00002AXHJUQA5', '0012A000023XlnqQAC', '0012A000023Y4bYQAS', '0012A000023YBFRQA4']
0012A000023Xo0OQAS ['0012A000023XleaQAC', '0012A000023YLw9QAG', '0012A000023XliRQAS', '0012A000023XtsCQAS']
0012A000023Y4bYQAS ['0012A000023XlnqQAC', '0012A00002AXHJUQA5', '0012A000023Y9gNQAS', '0012A000023XlkNQAS']
0012A000023XlkNQAS ['0012A000023Y4bYQAS', '0012A000023XlnqQAC', '0012A000023YDnWQAW', '0012A00002AXHJUQA5']
0012A000023XlnqQAC ['0012A00002AXHJUQA5', '0012A000023Y4bYQAS', '0012A000023Y9gNQAS', '0012A000023XlkNQAS']
0012A000023Xm35QAC ['0012A000023Xmd9QAC', '0012A000023XmgWQAS', '0012A000023Xm28QAC', '0012A000023XmDGQA0']
0012A000023XwMRQA0 ['0012A00

In [97]:
len(dd.keys())

29

In [110]:
result.head()
result.shape
set(result.id_accs.unique()) ^ set(test.id_accs.unique())
len(set(result.id_accs.unique()))

# result = result.append(pd.Series(list(set(result.id_accs.unique()) ^ set(test.id_accs.unique())), index=['id_accs']), ignore_index=True)
# result = result.append(list(set(result.id_accs.unique()) ^ set(test.id_accs.unique())), ignore_index=True)

id_accs            id_props
0  0012A000023YBFRQA4  a0I2A00000XQHqoUAH
1  0012A000023YBFRQA4  a0I2A00000XQIBCUA5
2  0012A000023YBFRQA4  a0I2A00000ZGRoRUAX
3  0012A000023YBFRQA4  a0I2A00000XQGR3UAP
4  0012A000023YBFRQA4  a0I2A00000XQGRcUAP

(236, 2)

set()

29

In [87]:
result.head(10)
# result = pd.concat([result, pd.Series(list(set(result.id_accs.unique()) ^ set(test.id_accs.unique())))], axis=0)

id_accs            id_props
2  0012A000023YBFRQA4  a0I2A00000ZGRoRUAX
3  0012A000023YBFRQA4  a0I2A00000XQGR3UAP
4  0012A000023YBFRQA4  a0I2A00000XQGRcUAP
5  0012A000023YBFRQA4  a0I2A00000XQGXpUAP
6  0012A000023YBFRQA4  a0I2A00000XQGXzUAP
7  0012A000023YBFRQA4  a0I2A00000XQGhVUAX
8  0012A000023YBFRQA4  a0I2A00000XQH1pUAH
9  0012A000023YBFRQA4  a0I2A00000XQH86UAH
0  0012A000023XlWaQAK  a0I2A00000XQJvZUAX
1  0012A000023XlWaQAK  a0I2A00000Y5k4wUAB

### Process Property File

In [34]:
##demo code
##get the acc property interaction which are in property files other unknown have not taken.
acc_prop.loc[acc_prop['id_props'].isin(list(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique())))].shape
##
aa = rm_unk.loc[rm_unk['id_accs'].isin(['0012A000023Xmd9QAC', '0012A000023XmgWQAS', '0012A000023Xm28QAC', '0012A000023XmDGQA0'])]
aa.groupby('id_accs').apply(lambda df: df.sample(1))['id_props'].values

(20865, 3)

array(['a0I2A00000XQM1yUAH', 'a0I2A00000XQM2KUAX', 'a0I2A00000XQKziUAH',
       'a0I2A00000XQLf7UAH'], dtype=object)

In [367]:
propert.loc[propert.sale_status=='N'].shape

(9137, 23)

In [368]:
##null check
propert.head()
propert.columns
propert.isnull().sum()

id_props building_status building_tax_expenses       city  \
0  a0I2A00000XQGKnUAP             NaN                   NaN  City-1975   
1  a0I2A00000XQGKtUAP             NaN                   NaN  City-0648   
2  a0I2A00000XQGKwUAP             NaN                   NaN  City-0424   
3  a0I2A00000XQGKxUAP             NaN                   NaN  City-0283   
4  a0I2A00000XQGKzUAP             NaN                   NaN  City-0981   

     class       county sale_status  portfolio       market  num_buildings  \
0  Class A  County-0465           Y       True  Market-0239            NaN   
1  Class A  County-0361           N      False  Market-0076            1.0   
2      NaN  County-0069           N      False  Market-0034            1.0   
3      NaN  County-0404           N      False  Market-0149            NaN   
4  Class B  County-0203           N      False  Market-0112           20.0   

      ...      price_per_sq_ft  price_per_unit  property_type_1  region__c  \
0     ...              1141.55      2109090.91    Multi-Housing  Southeast   
1     ...                  NaN             NaN    Multi-Housing       West   
2     ...                  NaN             NaN           Retail  Southwest   
3     ...                  NaN             NaN       Industrial  Southeast   
4     ...               126.06       106145.83    Multi-Housing  Southeast   

   sale_amount__c sale_date__c size_acres   size_sf size_units  year_built  
0     21440582.78   12/31/2016      19.60  508080.0      275.0      2014.0  
1             NaN          NaN       0.67   62524.0      100.0      2014.0  
2             NaN          NaN        NaN    8640.0        NaN      2012.0  
3             NaN          NaN        NaN  513734.0        NaN      2011.0  
4     45855000.00   10/31/2016      35.12  363747.0      432.0      1990.0  

[5 rows x 23 columns]

Index(['id_props', 'building_status', 'building_tax_expenses', 'city', 'class',
       'county', 'sale_status', 'portfolio', 'market', 'num_buildings',
       'num_floors', 'num_parking_spaces', 'occupancy_rate', 'price_per_sq_ft',
       'price_per_unit', 'property_type_1', 'region__c', 'sale_amount__c',
       'sale_date__c', 'size_acres', 'size_sf', 'size_units', 'year_built'],
      dtype='object')

id_props                    0
building_status          4108
building_tax_expenses    4488
city                        0
class                    3728
county                      0
sale_status                 0
portfolio                   0
market                      1
num_buildings            3710
num_floors               1954
num_parking_spaces       5310
occupancy_rate           1551
price_per_sq_ft          3167
price_per_unit           6695
property_type_1            60
region__c                   2
sale_amount__c           3222
sale_date__c             3060
size_acres               1862
size_sf                   430
size_units               5406
year_built                  0
dtype: int64

In [275]:
##replace nan with the highest valuein the column
propert['building_status'].fillna('Unknown',inplace=True)
propert['building_tax_expenses'].fillna('Unknown',inplace=True)
propert['class'].fillna('Unknown',inplace=True)
propert['sale_status'].fillna('Unknown',inplace=True)
propert['market'].fillna('Unknown',inplace=True)
propert['num_buildings'].fillna(1.0,inplace=True)
propert['num_floors'].fillna(1.0,inplace=True)
propert['num_parking_spaces'].fillna(200,inplace=True)
propert['occupancy_rate'].fillna(100,inplace=True)
propert['price_per_unit'].fillna(0,inplace=True)
propert['property_type_1'].fillna('Unknown',inplace=True)
propert['region__c'].fillna('Southwest',inplace=True)
propert['size_units'].fillna(1.0,inplace=True)
propert['year_built'].fillna(2000,inplace=True)

In [276]:
propert['price_per_sq_ft'].fillna(0,inplace=True)
propert['sale_amount__c'].fillna(0.0,inplace=True)
propert['sale_date__c'].fillna('Unknown',inplace=True)
propert['size_acres'].fillna(0.0,inplace=True)
propert['size_sf'].fillna(0.0,inplace=True)

In [39]:
# sns.countplot(x='year_built', data=propert)
propert['size_sf'].value_counts().head(10).plot.bar(width=.8)

In [277]:
##label encode the string data for the property file.
ls = ['building_status', 'building_tax_expenses', 'city', 'class', 'county', 'sale_status', 'portfolio', 'market',
     'property_type_1', 'region__c','sale_date__c']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in ls:
    le = preprocessing.LabelEncoder()
    propert[i] = le.fit_transform(propert[i])

In [282]:
propert.drop(['sale_year'],axis=1,inplace=True)

### knn on property file

In [372]:
##KNN to get 5 nearest neighbours
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(propert.iloc[:,1:], propert['id_props'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [283]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(propert.iloc[:,1:])
distances, indices = nbrs.kneighbors(propert.iloc[:,1:])

In [287]:
indices[5113:5115]
distances[5113:5114]
propert[propert.index == 1435]

array([[5113, 1435],
       [5114, 6164]], dtype=int64)

array([[     0.        ,  37932.99438071]])

id_props  building_status  building_tax_expenses  city  class  \
1435  a0I2A00000XQHKTUA5                0                   3201  1539      0   

      county  sale_status  portfolio  market  num_buildings     ...      \
1435     495            0          0     196            1.0     ...       

      price_per_sq_ft  price_per_unit  property_type_1  region__c  \
1435           314.18             0.0                6          5   

      sale_amount__c  sale_date__c  size_acres  size_sf  size_units  \
1435      21395000.0          1174       2.411  68098.0         1.0   

      year_built  
1435      2002.0  

[1 rows x 23 columns]

In [288]:
propert[propert.id_props == 'a0I2A00000XQK1YUAX']
propert[(propert.price_per_sq_ft<300) & (propert.price_per_sq_ft>150)]

id_props  building_status  building_tax_expenses  city  class  \
5113  a0I2A00000XQK1YUAX                0                   2758  1273      3   

      county  sale_status  portfolio  market  num_buildings     ...      \
5113     350            0          0     170            1.0     ...       

      price_per_sq_ft  price_per_unit  property_type_1  region__c  \
5113           202.47             0.0                8          4   

      sale_amount__c  sale_date__c  size_acres   size_sf  size_units  \
5113      21400000.0          1191       9.661  105696.0         1.0   

      year_built  
5113      1999.0  

[1 rows x 23 columns]

id_props  building_status  building_tax_expenses  city  class  \
10    a0I2A00000XQGLCUA5                3                   3875  1154      3   
35    a0I2A00000XQGM3UAP                0                   2574  1336      0   
37    a0I2A00000XQGM8UAP                0                   3342   721      0   
42    a0I2A00000XQGMJUA5                0                   3841    72      0   
45    a0I2A00000XQGMOUA5                0                   3664   411      3   
59    a0I2A00000XQGMmUAP                3                   3875  1311      3   
61    a0I2A00000XQGMpUAP                0                   3758   807      1   
74    a0I2A00000XQGN6UAP                0                   2897   795      1   
81    a0I2A00000XQGNLUA5                0                    343   355      1   
90    a0I2A00000XQGNoUAP                0                   3157  1359      0   
92    a0I2A00000XQGNsUAP                0                   1151   293      2   
97    a0I2A00000XQGO2UAP                0                   1050   193      0   
98    a0I2A00000XQGO3UAP                0                   3163    20      0   
109   a0I2A00000XQGOOUA5                3                   3875  1429      3   
111   a0I2A00000XQGOUUA5                0                   1620  1762      3   
112   a0I2A00000XQGOYUA5                0                   1650  1776      3   
116   a0I2A00000XQGOdUAP                3                   3875   630      3   
120   a0I2A00000XQGOnUAP                0                    340   721      3   
130   a0I2A00000XQGPIUA5                3                   3875  1200      0   
133   a0I2A00000XQGPPUA5                0                      1  1507      1   
142   a0I2A00000XQGPlUAP                0                   2840   243      3   
143   a0I2A00000XQGPmUAP                0                   3437   601      3   
149   a0I2A00000XQGQ0UAP                0                   2242   853      3   
157   a0I2A00000XQGQHUA5                0                   2943  1009      0   
172   a0I2A00000XQGR5UAP                0                   3219  1607      0   
173   a0I2A00000XQGR6UAP                0                    378  1058      3   
177   a0I2A00000XQGRDUA5                0                   3811   721      0   
188   a0I2A00000XQGRZUA5                0                   3456  1249      1   
194   a0I2A00000XQGRzUAP                0                   3875   390      0   
201   a0I2A00000XQGSHUA5                0                   2704   822      2   
...                  ...              ...                    ...   ...    ...   
8676  a0I2A00000XwaFZUAZ                0                   1586   745      1   
8684  a0I2A00000XwbBpUAJ                3                   3875  1429      3   
8686  a0I2A00000XwbP0UAJ                3                   3875    72      0   
8688  a0I2A00000XwcT3UAJ                3                   3875   995      0   
8689  a0I2A00000XwcsFUAR                3                   3875  1657      3   
8695  a0I2A00000XwfdjUAB                3                   3875   868      0   
8697  a0I2A00000XwgliUAB                0                   2399  1432      3   
8701  a0I2A00000XwqFcUAJ                0                   3812  1720      2   
8728  a0I2A00000XyIgiUAF                0                   2874    44      1   
8830  a0I2A00000Y6DYVUA3                3                   3875   156      0   
8840  a0I2A00000Y6DswUAF                3                   3875   286      0   
8859  a0I2A00000Y6PBtUAN                3                   3875  1119      3   
8904  a0I2A00000Y6b1NUAR                3                   3875   888      0   
8925  a0I2A00000Y7POaUAN                3                   3875  1259      3   
8926  a0I2A00000Y7PObUAN                3                   3875  1259      3   
8936  a0I2A00000YFYKYUA5                0                    871   847      0   
8949  a0I2A00000YG1PIUA1                2                   2091    72      3   
8983  a0I2A00000YH3HoUAL                0      

In [373]:
##set the index value as row for test to get from accounts table.
propert.reset_index(inplace = True)
index1 = propert['index']

In [374]:
##get all values from ds and make same predictions for all the same.
kk=[]
for i in ds.values():
    for j in i:
        kk.append(j)
len(set(kk))
# test.head()

69

In [375]:
index1 = propert['index'].loc[propert['id_props'].isin(kk)].values

In [376]:
index1

array([ 115,  393,  515,  552,  635,  702,  944, 1088, 1160, 1304, 1371,
       1377, 1447, 1604, 2122, 2530, 2891, 2911, 3203, 3428, 3889, 4000,
       4202, 4333, 4425, 4473, 4774, 4918, 5057, 5352, 5438, 5644, 5774,
       5989, 5994, 6004, 6315, 6335, 6371, 6406, 6566, 6589, 6663, 6732,
       6754, 7197, 7250, 7303, 7308, 7315, 7349, 7354, 7552, 7808, 7883,
       7923, 8003, 8144, 8161, 8216, 8245, 8436, 8456, 8464, 8873, 9158,
       9387, 9434, 9965], dtype=int64)

In [377]:
##get top 5 neighbours
top5ids = neigh.kneighbors(propert.iloc[index1,2:])[1]
print(neigh.kneighbors(propert.iloc[index1,2:])[1])

[[ 115 7379 7374 7989 4262]
 [ 393  642 2041   26 2852]
 [ 515  979 7699 1056 2176]
 [ 552 3303 1010 7925 2353]
 [ 635 8506 1570 4529 8539]
 [ 702  899 1376  540 3132]
 [ 944  844 2437 5818 8322]
 [1088 2177 7011 8737 5644]
 [1160 5590 7213 1451 8813]
 [1304 6583  433 2255  431]
 [1371 9210 8984 5259 7675]
 [1377 6919 4900  668 5963]
 [1447 3071 6966 6631 1736]
 [1604 2685 2846 2465 4633]
 [2122 7139 2165 4014 8987]
 [2530 7281 9443 7476 8762]
 [2891 5518 4133 2142 8032]
 [2911 8618 7641 1510 7751]
 [3203 7318 4167 3709 4215]
 [3428 3634 3644 5982 1903]
 [3889 5413 4099  716 4807]
 [4000 9051 6297 9547 8901]
 [4202 3002 8667 9071 1695]
 [4333 2260 8698  934 3478]
 [4425 4470 7462 5001 1332]
 [4473 7464 2414 5997 1922]
 [4774 3108 7747 3869 3270]
 [4918 1536 5374  629 3923]
 [5057 7803 6740 3635 9536]
 [5352 7362 6819 2658 2006]
 [5438 5283 1484 9124 2341]
 [5644 7011 2177 1088 8737]
 [5774 7366 2158 1786 6197]
 [5989 3442 6749 3426 3486]
 [5994 2910 7742 3455 5855]
 [6004 5245  738 972

In [378]:
propert.head()

index            id_props  building_status  building_tax_expenses  city  \
0      0  a0I2A00000XQGKnUAP                3                   4862  1491   
1      1  a0I2A00000XQGKtUAP                3                   4862   483   
2      2  a0I2A00000XQGKwUAP                3                   4862   322   
3      3  a0I2A00000XQGKxUAP                3                   4862   209   
4      4  a0I2A00000XQGKzUAP                3                   4862   727   

   class  county  sale_status  portfolio  market     ...      price_per_sq_ft  \
0      0     384            1          1     218     ...              1141.55   
1      0     297            0          0      66     ...                 0.00   
2      3      57            0          0      27     ...                 0.00   
3      3     333            0          0     135     ...                 0.00   
4      1     166            0          0      99     ...               126.06   

   price_per_unit  property_type_1  region__c  sale_amount__c  sale_date__c  \
0      2109090.91                5          3     21440582.78           852   
1            0.00                5          5            0.00          2483   
2            0.00                8          4            0.00          2483   
3            0.00                2          3            0.00          2483   
4       106145.83                5          3     45855000.00           390   

   size_acres   size_sf  size_units  year_built  
0       19.60  508080.0       275.0      2014.0  
1        0.67   62524.0       100.0      2014.0  
2        0.00    8640.0         1.0      2012.0  
3        0.00  513734.0         1.0      2011.0  
4       35.12  363747.0       432.0      1990.0  

[5 rows x 24 columns]

In [379]:
##index and id_accs dictionary 
dic_acc = {}
k = 0
all_account = propert['id_props']
for i in all_account:
    dic_acc[k] = i
    k = k+1

In [381]:
##get the test id's as key and 5 neighbour's id's as value.
dd1={}
fir = list(top5ids[:,0:1].reshape(-1))
sec = list(top5ids[:,1:2].reshape(-1))
thr  =list(top5ids[:,2:3].reshape(-1))
four  =list(top5ids[:,3:4].reshape(-1))
fif = list(top5ids[:,4:5].reshape(-1))
for i in range(69):
    #accounts['id_accs'].loc[test_acc['index'].isin([4])]
    dd1[dic_acc[fir[i]]] = [dic_acc[sec[i]], dic_acc[thr[i]], dic_acc[four[i]], dic_acc[fif[i]]]

In [382]:
len(dd1.keys())

69

In [82]:
##value is not there
dd1['a0I2A00000ZGhcMUAT']

KeyError: 'a0I2A00000ZGhcMUAT'

In [383]:
##link both ds and dd1 dictionary
dq={}
for i,j in ds.items():
    dq[i]=[]
    for k in j:
        if 'a0I2A00000ZGhcMUAT' == k:
            continue
        val = dq[i]
        val.extend(dd1[k])
        dq[i] = val

In [384]:
lk1 = []
lk2 = []
for i,j in dq.items():
    if len(j)==0:
        lk1.extend([i] * 1)
        lk2.extend([''])
    else:
        lk1.extend([i] * len(j))
        lk2.extend(j)

In [385]:
len(lk1)
len(lk2)
sub = pd.DataFrame({'id_accs': lk1,'id_prop': lk2})
sub.to_csv('submis.csv',index=False)

298

298

In [104]:
len(set(dq.keys()))
len(set(lk1))

29

29

### Submission file Prep

In [386]:
##property id's which not are in account_to_property table
prop_free = propert.loc[propert['id_props'].isin(set(propert.id_props.unique()).intersection(acc_prop.id_props.unique()))]
prop_free.shape

(8167, 24)

In [387]:
##get all values from ds and make same predictions for all the same.
kk=[]
for i in dd1.values():
    for j in i:
        kk.append(j)
len(set(kk))
# test.head()

268

In [391]:
prop_ff = prop_free.loc[prop_free['id_props'].isin(set(prop_free.id_props.unique()).intersection(set(kk)))]
# prop_ff = propert.loc[deal_prop['id_props'].isin(set(deal_prop.id_props.unique()).intersection(set(kk)))]
# [60:150] = 0.00148
# [150:] = 0.00182
b = list(prop_ff['id_props'].values)[150:]
a = list(test.id_accs.values)
len(a)
l1=[]
l2=[]
for i in a:
    for j in set(b):
        l1.append(i)
        l2.append(j)
len(l1)
len(l2)
sub = pd.DataFrame({'id_accs': l1,'id_prop': l2})
sub.to_csv('submis.csv',index=False)

29

2262

2262

## Approach & Results:
#### 1. Apply knn on account file to get the nearest neighbour for the test id's & then take those predicted account's and get there corresponding id's for property from "account to property" file.Now apply the knn on the property id's mapped from first step and get their nearest neighbour.predict for all test id's these same group id's came from knn.
#### 2. property id's which are in property table but not are in account_to_property table giving result as 0.

### Interaction Matrix

In [41]:
columns = list(account.columns)
columns.remove('id_accs')
acc_ = pd.DataFrame()
for col in columns:
    sub_ = pd.get_dummies(account[col],prefix=col,dummy_na=True)
    acc_ = pd.concat([acc_,sub_],axis=1)
    acc_.reset_index(drop=True,inplace=True)

In [ ]:
columns = list(propert.columns)
columns.remove('id_props')
prop_ = pd.DataFrame()
for col in columns:
    sub1_ = pd.get_dummies(propert[col],prefix=col,dummy_na=True)
    prop_ = pd.concat([prop_,sub1_],axis=1)
    prop_.reset_index(drop=True,inplace=True)

In [9]:
account.loc[account.id_accs == '0012A000023XsHdQAK']
test.loc[test.id_accs == '0012A000023YBFRQA4']

id_accs  active_deals  activity_count  buyer_book  \
511  0012A000023XsHdQAK           437           11308        True   

     servicing_contract           investor_type   cmbs  consultant  \
511                True  Life Insurance Company  False       False   

     correspondent  foreign  master_servicer  lender_book  loan_sales_book  \
511          False    False            False         True             True   

     loan_servicing  num_deals_as_client  num_deals_as_investor  \
511           False                    4                    433   

     number_of_properties  number_of_related_deals  \
511                     3                      456   

     number_of_related_properties  number_of_won_deals_as_client  
511                           148                              4

id_accs
15  0012A000023YBFRQA4

In [40]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(account.iloc[:1,1:],account.iloc[2:3,1:])
1-cosine(account.iloc[:1,1:],account.iloc[2:3,1:])

array([[ 0.99917328]])

0.9991732764936081

In [36]:
account.iloc[:3,1:]
account.iloc[:1,1:]
account.iloc[1:2,1:]

active_deals  activity_count  buyer_book  servicing_contract  \
0             1            2367           1                   0   
1             9            7974           1                   0   
2           891           50161           1                   1   

   investor_type  cmbs  consultant  correspondent  foreign  master_servicer  \
0             10     0           0              0        0                0   
1             13     0           0              0        0                0   
2              3     0           0              0        0                0   

   lender_book  loan_sales_book  loan_servicing  num_deals_as_client  \
0            0                0               0                    0   
1            1                1               0                    7   
2            1                1               0                   40   

   num_deals_as_investor  number_of_properties  number_of_related_deals  \
0                      1                     1                        2   
1                      2                    77                       18   
2                    851                   693                      996   

   number_of_related_properties  number_of_won_deals_as_client  
0                             1                              0  
1                            85                              7  
2                          1167                             37

active_deals  activity_count  buyer_book  servicing_contract  \
0             1            2367           1                   0   

   investor_type  cmbs  consultant  correspondent  foreign  master_servicer  \
0             10     0           0              0        0                0   

   lender_book  loan_sales_book  loan_servicing  num_deals_as_client  \
0            0                0               0                    0   

   num_deals_as_investor  number_of_properties  number_of_related_deals  \
0                      1                     1                        2   

   number_of_related_properties  number_of_won_deals_as_client  
0                             1                              0

active_deals  activity_count  buyer_book  servicing_contract  \
1             9            7974           1                   0   

   investor_type  cmbs  consultant  correspondent  foreign  master_servicer  \
1             13     0           0              0        0                0   

   lender_book  loan_sales_book  loan_servicing  num_deals_as_client  \
1            1                1               0                    7   

   num_deals_as_investor  number_of_properties  number_of_related_deals  \
1                      2                    77                       18   

   number_of_related_properties  number_of_won_deals_as_client  
1                            85                              7

In [24]:
os.chdir(r'C:\Users\royal\Downloads\bb020fc4-9-Dataset\Dataset\Property dataset')
qw = pd.read_csv('qwer.csv')
subm=pd.read_csv('prev_sub_fail.csv')

In [288]:
propert[~propert.id_props.isin(list(set(subm.id_prop.unique())))].shape

(10015, 23)

In [285]:
propert.shape

(10206, 23)

In [178]:
subm.head(30)

id_accs             id_prop
0   0012A00002AXHJUQA5  a0I2A00000XQGjJUAX
1   0012A00002AXHJUQA5  a0I2A00000XQGkjUAH
2   0012A00002AXHJUQA5  a0I2A00000XQGotUAH
3   0012A00002AXHJUQA5  a0I2A00000XQGsoUAH
4   0012A00002AXHJUQA5  a0I2A00000XQGt3UAH
5   0012A00002AXHJUQA5  a0I2A00000XQGzFUAX
6   0012A00002AXHJUQA5  a0I2A00000XQGzpUAH
7   0012A00002AXHJUQA5  a0I2A00000XQH0TUAX
8   0012A00002AXHJUQA5  a0I2A00000XQH3gUAH
9   0012A00002AXHJUQA5  a0I2A00000XQH5zUAH
10  0012A00002AXHJUQA5  a0I2A00000XQH9YUAX
11  0012A00002AXHJUQA5  a0I2A00000XQHCUUA5
12  0012A00002AXHJUQA5  a0I2A00000XQHE0UAP
13  0012A00002AXHJUQA5  a0I2A00000XQHFsUAP
14  0012A00002AXHJUQA5  a0I2A00000XQHReUAP
15  0012A00002AXHJUQA5  a0I2A00000XQHSzUAP
16  0012A00002AXHJUQA5  a0I2A00000XQHTfUAP
17  0012A00002AXHJUQA5  a0I2A00000XQHTgUAP
18  0012A00002AXHJUQA5  a0I2A00000XQHVbUAP
19  0012A00002AXHJUQA5  a0I2A00000XQHXIUA5
20  0012A00002AXHJUQA5  a0I2A00000XQHYQUA5
21  0012A00002AXHJUQA5  a0I2A00000XQHaTUAX
22  0012A00002AXHJUQA5  a0I2A00000XQHdFUAX
23  0012A00002AXHJUQA5  a0I2A00000XQHngUAH
24  0012A00002AXHJUQA5  a0I2A00000XQHpaUAH
25  0012A00002AXHJUQA5  a0I2A00000XQHtZUAX
26  0012A00002AXHJUQA5  a0I2A00000XQHzBUAX
27  0012A00002AXHJUQA5  a0I2A00000XQI0JUAX
28  0012A00002AXHJUQA5  a0I2A00000XQIBqUAP
29  0012A00002AXHJUQA5  a0I2A00000XQISAUA5

In [237]:
##submit same properties for all the test accounts.
def sub_file_all_same(l1):
    lk1 = []
    lk2 = []
    for i in test.id_accs.values:
        lk1.extend([i] * len(l1))
        lk2.extend(l1)
    sub = pd.DataFrame({'id_accs': lk1,'id_prop': lk2})
    sub.to_csv('uts.csv',index=False)
    return len(lk1)

In [175]:
sub_file_all_same(l1)

5829

In [182]:
pd.concat({k: pd.Series(v) for k, v in dd.items()})

0012A000023XlWaQAK  0    0012A000023XpKsQAK
                    1    0012A000023XmRwQAK
                    2    0012A000023XonpQAC
                    3    0012A000023XlbuQAC
0012A000023XlbuQAC  0    0012A000023XonpQAC
                    1    0012A000023XpKsQAK
                    2    0012A000023XlWaQAK
                    3    0012A000023XmRwQAK
0012A000023Xlh9QAC  0    0012A000023XsHdQAK
                    1    0012A000023XvtGQAS
                    2    0012A000023Xt67QAC
                    3    0012A000023XsS6QAK
0012A000023XlhdQAC  0    0012A000023Xlp3QAC
                    1    0012A000023Xpn2QAC
                    2    0012A000023YB67QAG
                    3    0012A000023XljZQAS
0012A000023XlkNQAS  0    0012A000023Y4bYQAS
                    1    0012A000023XlnqQAC
                    2    0012A000023YDnWQAW
                    3    0012A00002AXHJUQA5
0012A000023XlnWQAS  0    0012A000023XrTNQA0
                    1    0012A000029wHvgQAE
                    2    0012A00

In [195]:
propert[propert.id_props.isin(set(acc_prop.loc[acc_prop['id_accs'].isin(list(j))].id_props.values))]

id_props building_status  \
139   a0I2A00000XQGPbUAP        Proposed   
280   a0I2A00000XQGUnUAP        Existing   
321   a0I2A00000XQGWEUA5             NaN   
352   a0I2A00000XQGXOUA5             NaN   
399   a0I2A00000XQGZkUAP             NaN   
476   a0I2A00000XQGdDUAX             NaN   
582   a0I2A00000XQGj9UAH             NaN   
605   a0I2A00000XQGk1UAH             NaN   
608   a0I2A00000XQGk4UAH        Existing   
611   a0I2A00000XQGkAUAX             NaN   
620   a0I2A00000XQGkbUAH        Existing   
727   a0I2A00000XQGowUAH        Existing   
735   a0I2A00000XQGpLUAX        Existing   
736   a0I2A00000XQGpMUAX        Existing   
737   a0I2A00000XQGpNUAX        Existing   
738   a0I2A00000XQGpPUAX        Existing   
739   a0I2A00000XQGpRUAX        Existing   
748   a0I2A00000XQGq2UAH        Existing   
754   a0I2A00000XQGqNUAX             NaN   
756   a0I2A00000XQGqTUAX             NaN   
778   a0I2A00000XQGrEUAX             NaN   
799   a0I2A00000XQGrzUAH             NaN   
837   a0I2A00000XQGtGUAX             NaN   
938   a0I2A00000XQGxZUAX             NaN   
974   a0I2A00000XQGzUUAX             NaN   
984   a0I2A00000XQGzoUAH             NaN   
996   a0I2A00000XQH0DUAX        Existing   
1000  a0I2A00000XQH0RUAX             NaN   
1019  a0I2A00000XQH1OUAX        Existing   
1025  a0I2A00000XQH1aUAH        Existing   
...                  ...             ...   
7703  a0I2A00000XQLosUAH             NaN   
7722  a0I2A00000XQLpbUAH             NaN   
7905  a0I2A00000XQLy2UAH             NaN   
7957  a0I2A00000XQLzzUAH             NaN   
7985  a0I2A00000XQM16UAH        Existing   
8010  a0I2A00000XQM2YUAX        Existing   
8019  a0I2A00000XQM2xUAH        Existing   
8026  a0I2A00000XQM3LUAX        Existing   
8047  a0I2A00000XQM45UAH        Existing   
8062  a0I2A00000XQM4kUAH        Existing   
8072  a0I2A00000XQM55UAH        Existing   
8078  a0I2A00000XQM5NUAX             NaN   
8090  a0I2A00000XQM6EUAX             NaN   
8127  a0I2A00000XQM8FUAX        Existing   
8132  a0I2A00000XQM8TUAX        Existing   
8187  a0I2A00000XQMBZUA5        Existing   
8208  a0I2A00000XQQFyUAP        Existing   
8209  a0I2A00000XQQGSUA5        Existing   
8239  a0I2A00000XQWyZUAX        Existing   
8387  a0I2A00000XuMiVUAV        Existing   
8393  a0I2A00000XuNNnUAN        Existing   
8432  a0I2A00000XuU3wUAF        Existing   
8457  a0I2A00000XuXjVUAV        Existing   
8751  a0I2A00000Y5k52UAB             NaN   
8752  a0I2A00000Y5k54UAB             NaN   
8980  a0I2A00000YH03hUAD        Existing   
8981  a0I2A00000YH0JIUA1        Existing   
9218  a0I2A00000ZGGvJUAX             NaN   
9288  a0I2A00000ZGZEnUAP             NaN   
9312  a0I2A00000ZGdbIUAT             NaN   

                                  building_tax_expenses       city    class  \
139   2008 Combined Tax/Ops @ $5.83/sf; 2007 Combine...  City-0960  Class A   
280                              2012 Tax @ $17.16/Unit  City-0526  Class A   
321                                                 NaN  City-1860      NaN   
352                                                 NaN  City-0376  Class A   
399                                                 NaN  City-1415  Class A   
476                                                 NaN  City-2120      NaN   
582                                                 NaN  City-0376      NaN   
605                                                 NaN  City-0618      NaN   
608                             2011 Tax @ $423.56/Unit  City-1207  Class B   
611                                                 NaN  City-1186      NaN   
620   2016 Tax @ $3.88/sf, 2010 Est Tax @ $1.44/sf; ...  City-0969  Class A   
727                                 2016 Tax @ $1.27/sf  City-0936  Class A   
735                                 2016 Tax @ $1.33/sf  City-0936  Class B   
736                                 2012 Tax @ $0.97/sf  City-0936  Class B   
737                                 2016 Tax @ $1.02/sf  City-0936  Class C   
738                    

In [196]:
lk1 = []
lk2 = []
for i,j in dd.items():
    ll = list(propert[propert.id_props.isin(set(acc_prop.loc[acc_prop['id_accs'].isin(list(j))].id_props.values))].id_props.values)
    if len(ll)==0:
        lk1.extend([i] * 1)
        lk2.extend([''])
    else:
        lk1.extend([i] * len(ll))
        lk2.extend(ll)

In [197]:
sub = pd.DataFrame({'id_accs': lk1,'id_prop': lk2})
sub.to_csv('qwer.csv',index=False)

In [198]:
len(lk2)

1187

In [142]:
dd['0012A000023XlWaQAK']
list(dd['0012A000023XlWaQAK'])

array(['a0I2A00000XQHJ7UAP', 'a0I2A00000XQLqDUAX'], dtype=object)

['a0I2A00000XQHJ7UAP', 'a0I2A00000XQLqDUAX']

In [171]:
lk1 = []
lk2 = []
for i,j in dd.items():
    ll = list(j)
    if len(ll)==0:
        lk1.extend([i] * 1)
        lk2.extend([''])
    else:
        lk1.extend([i] * len(ll))
        lk2.extend(ll)

In [172]:
len(lk1)

103

In [174]:
ll1 = []
for i in dd.values():
    ll1.extend(list(i))

In [199]:
sub_file_all_same(lk2)

34423

In [246]:
account[account.id_accs=='0012A000023YDnWQAW']

id_accs  active_deals  activity_count  buyer_book  \
11  0012A000023YDnWQAW           606          217301        True   

    servicing_contract           investor_type   cmbs  consultant  \
11                True  Life Insurance Company  False       False   

    correspondent  foreign  master_servicer  lender_book  loan_sales_book  \
11          False    False            False         True             True   

    loan_servicing  num_deals_as_client  num_deals_as_investor  \
11           False                   51                    555   

    number_of_properties  number_of_related_deals  \
11                   108                      664   

    number_of_related_properties  number_of_won_deals_as_client  
11                           381                             38

In [245]:
propert[propert.id_props.isin(['a0I2A00000XQK1YUAX','a0I2A00000XQLImUAP'])]

id_props building_status building_tax_expenses       city  \
5113  a0I2A00000XQK1YUAX        Existing   2016 Tax @ $2.27/sf  City-1606   
6997  a0I2A00000XQLImUAP             NaN                   NaN  City-1510   

     class       county sale_status  portfolio       market  num_buildings  \
5113   NaN  County-0400           N      False  Market-0178            1.0   
6997   NaN  County-0464           N       True  Market-0169            NaN   

        ...      price_per_unit  property_type_1  region__c  sale_amount__c  \
5113    ...                 NaN           Retail  Southwest      21400000.0   
6997    ...                 NaN           Retail  Southwest             NaN   

      sale_date__c size_acres   size_sf  size_units year_built  sale_year  
5113     4/23/2008      9.661  105696.0         NaN     1999.0     2008.0  
6997           NaN        NaN  161568.0         NaN     2010.0        NaN  

[2 rows x 24 columns]

In [213]:
acc_prop[acc_prop.id_props == 'a0I2A00000XQLImUAP']

Empty DataFrame
Columns: [id, id_accs, id_props]
Index: []

In [247]:
oppor[oppor.id_deals.isin(list(deal_prop[deal_prop.id_props.isin(['a0I2A00000XQLImUAP','a0I2A00000XQK1YUAX'])].id_deals.values))]
deal_prop[deal_prop.id_props == 'a0I2A00000XQLImUAP']
deal_prop[deal_prop.id_props == 'a0I2A00000XQK1YUAX']
# deal_prop[deal_prop.id_props == 'a0I2A00000XQIdNUAX']
account[account.id_accs.isin(list(oppor[oppor.id_deals.isin(list(deal_prop[deal_prop.id_props.isin(['a0I2A00000XQLImUAP','a0I2A00000XQK1YUAX'])].id_deals.values))].id_accs.values))]

id_accs accounting_date  best_initial_bid  \
22558  0012A000029wIP5QAM       5/15/2017               0.0   
25001  0012A000026WVuDQAW       3/31/2017               0.0   

                 id_deals   closedate     date_closed   deal_type  \
22558  0062A00000nESasQAG  12/16/2016  5/11/2017 0:00  Fixed-rate   
25001  0062A00000moEmzQAE  10/14/2016  3/31/2017 0:00  Fixed-rate   

       deal_update_flag  debt_yield  fiscal  fiscalquarter  fiscalyear  \
22558              True         0.0  2016 4              4        2016   
25001              True         0.0  2016 4              4        2016   

      platform property_group property_type  
22558     Debt         Retail        Retail  
25001     Debt         Retail        Retail

id            id_deals            id_props
18567  a0A2A00000TzqKvUAJ  0062A00000nESasQAG  a0I2A00000XQLImUAP

id            id_deals            id_props
14184  a0A2A00000TzoprUAB  0062A00000moEmzQAE  a0I2A00000XQK1YUAX

id_accs  active_deals  activity_count  buyer_book  \
2591  0012A000029wIP5QAM            30            9332       False   

      servicing_contract investor_type   cmbs  consultant  correspondent  \
2591               False          REIT  False       False          False   

      foreign  master_servicer  lender_book  loan_sales_book  loan_servicing  \
2591    False            False        False            False           False   

      num_deals_as_client  num_deals_as_investor  number_of_properties  \
2591                   29                      1                    73   

      number_of_related_deals  number_of_related_properties  \
2591                       40                            75   

      number_of_won_deals_as_client  
2591                             28

In [270]:
len(set(deal_prop[deal_prop.id_deals.isin(list(oppor.id_deals.values))].id))
len(set(deal_prop[deal_prop.id_props.isin(list(propert.id_props.values))].id))
aa1 = list(set(deal_prop[deal_prop.id_props.isin(list(propert.id_props.values))].id_props) - set(deal_prop[deal_prop.id_deals.isin(list(oppor.id_deals.values))].id_props))

bb1 = list(set(aa1) - set(acc_prop.id_props))

17771

12013

In [274]:
cc1 = propert[propert.id_props.isin(aa1)]
cc1[cc1.id_props == 'a0I2A00000XQK1YUAX']

Empty DataFrame
Columns: [id_props, building_status, building_tax_expenses, city, class, county, sale_status, portfolio, market, num_buildings, num_floors, num_parking_spaces, occupancy_rate, price_per_sq_ft, price_per_unit, property_type_1, region__c, sale_amount__c, sale_date__c, size_acres, size_sf, size_units, year_built, sale_year]
Index: []

[0 rows x 24 columns]

In [254]:
# account[account.id_accs.isin(list(oppor[oppor.id_deals.isin(list(deal_prop[deal_prop.id_props.isin(list(propert.id_props.values))].id_deals.values))].id_accs.values))]
deal_prop[deal_prop.id_deals.isin(list(oppor.id_deals.values))].subtract(deal_prop[deal_prop.id_props.isin(list(propert.id_props.values))])
deal_prop[deal_prop.id_props.isin(list(propert.id_props.values))]

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [291]:
pp = oppor[(oppor.fiscalyear==2016) & (oppor.property_group == 'Retail') & (oppor.platform == 'Debt') & (oppor.deal_update_flag == True)]

In [297]:
deal_prop[deal_prop.id_deals.isin(list(pp.id_deals.values))].shape
account[account.id_accs.isin(list(pp.id_accs.values))].shape

pp[pp.id_accs.isin(list(account.id_accs.values))].shape

deal_prop[deal_prop.id_deals.isin(list(pp.id_deals.values))](pp[pp.id_accs.isin(list(account.id_accs.values))])

(184, 3)

(48, 20)

(91, 15)

accounting_date  best_initial_bid closedate date_closed deal_type  \
1231              NaN               NaN       NaN         NaN       NaN   
2576              NaN               NaN       NaN         NaN       NaN   
2609              NaN               NaN       NaN         NaN       NaN   
2910              NaN               NaN       NaN         NaN       NaN   
2918              NaN               NaN       NaN         NaN       NaN   
2927              NaN               NaN       NaN         NaN       NaN   
2928              NaN               NaN       NaN         NaN       NaN   
2937              NaN               NaN       NaN         NaN       NaN   
2952              NaN               NaN       NaN         NaN       NaN   
2963              NaN               NaN       NaN         NaN       NaN   
2996              NaN               NaN       NaN         NaN       NaN   
3000              NaN               NaN       NaN         NaN       NaN   
3009              NaN               NaN       NaN         NaN       NaN   
4016              NaN               NaN       NaN         NaN       NaN   
4436              NaN               NaN       NaN         NaN       NaN   
4582              NaN               NaN       NaN         NaN       NaN   
4584              NaN               NaN       NaN         NaN       NaN   
5057              NaN               NaN       NaN         NaN       NaN   
5294              NaN               NaN       NaN         NaN       NaN   
5480              NaN               NaN       NaN         NaN       NaN   
5671              NaN               NaN       NaN         NaN       NaN   
6597              NaN               NaN       NaN         NaN       NaN   
6727              NaN               NaN       NaN         NaN       NaN   
7311              NaN               NaN       NaN         NaN       NaN   
7557              NaN               NaN       NaN         NaN       NaN   
8008              NaN               NaN       NaN         NaN       NaN   
8321              NaN               NaN       NaN         NaN       NaN   
8346              NaN               NaN       NaN         NaN       NaN   
8387              NaN               NaN       NaN         NaN       NaN   
8433              NaN               NaN       NaN         NaN       NaN   
...               ...               ...       ...         ...       ...   
24405             NaN               NaN       NaN         NaN       NaN   
24556             NaN               NaN       NaN         NaN       NaN   
24572             NaN               NaN       NaN         NaN       NaN   
24582             NaN               NaN       NaN         NaN       NaN   
24586             NaN               NaN       NaN         NaN       NaN   
24675             NaN               NaN       NaN         NaN       NaN   
24713             NaN               NaN       NaN         NaN       NaN   
24740             NaN               NaN       NaN         NaN       NaN   
24754             NaN               NaN       NaN         NaN       NaN   
24860             NaN               NaN       NaN         NaN       NaN   
24863             NaN               NaN       NaN         NaN       NaN   
24987             NaN               NaN       NaN         NaN       NaN   
24995             NaN               NaN       NaN         NaN       NaN   
25004             NaN               NaN       NaN         NaN       NaN   
25030             NaN               NaN       NaN         NaN       NaN   
25033             NaN               NaN       NaN         NaN       NaN   
25040             NaN               NaN       NaN         NaN       NaN   
25050             NaN               NaN       NaN         NaN       NaN   
25054             NaN               NaN       NaN         NaN       NaN   
25055             NaN               NaN       NaN         NaN       NaN   
25056             NaN               NaN       NaN         NaN       NaN   
25077             NaN          

In [240]:
os.chdir(r'C:\Users\royal\Downloads')
#,'a0I2A00000XQIdNUAX'
sub_file_all_same(['a0I2A00000XQLImUAP','a0I2A00000XQK1YUAX'])

58

In [337]:
sub_file_all_same(list(set(deal_prop[deal_prop.id_deals.isin(list(pp.id_deals.values))][110:111].id_props.values)))

29

In [303]:
q= deal_prop[deal_prop.id_deals.isin(list(pp.id_deals.values))]
q[q.id_props == 'a0I2A00000XQK1YUAX']

id            id_deals            id_props
14184  a0A2A00000TzoprUAB  0062A00000moEmzQAE  a0I2A00000XQK1YUAX